## Installation

In [9]:
# %pip install google-genai

In [10]:
import pandas as pd
import re

In [11]:
material_type = ["06"]

In [12]:
mm = pd.read_csv("Vw_MaterialMaster.csv", encoding="utf-8")

# แสดงข้อมูล 5 แถวแรก
print(mm.head())

   Material_Type Material_Number                      Material_Description  \
0            3.0      7803141642          SW,PROXI,3RG4014-0AG30-Z,SIEMENS   
1           32.0      7832234005  FELT,PICK UP PM#4,5,23.00X3.95 M,ATROVIT   
2            3.0     75030202210            CONTACTOR,CA 501, 1NC,ABB  [5]   
3            3.0     75030202220            CONTACTOR,CA 510, 1NO,ABB  [5]   
4            3.0     75030202260      CONTACTOR,CAL 511,1NO 1NC.,ABB  [ZN]   

  PlantID Plant Plant_Description UOM  Moving_avg_price  \
0   PT014  7811      TPC-Ban Pong  PC           5400.00   
1   PT014  7811      TPC-Ban Pong  PC         314484.73   
2   PT011  7560    SKIC-Wangsa-la  ST            124.00   
3   PT011  7560    SKIC-Wangsa-la  ST            127.00   
4   PT011  7560    SKIC-Wangsa-la  ST            250.00   

   Planned_delivery_time  Reorder_point  ...  CompID CompanyCode  \
0                    NaN            2.0  ...   CP005         780   
1                    NaN            0.0 

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_7444/2540408686.py:1: DtypeWarning: Columns (4,11,14,15,16,17,18,19,20,26,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  mm = pd.read_csv("Vw_MaterialMaster.csv", encoding="utf-8")


In [13]:
mm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310160 entries, 0 to 310159
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Material_Type          310159 non-null  float64
 1   Material_Number        310160 non-null  object 
 2   Material_Description   310160 non-null  object 
 3   PlantID                310160 non-null  object 
 4   Plant                  310160 non-null  object 
 5   Plant_Description      310160 non-null  object 
 6   UOM                    310020 non-null  object 
 7   Moving_avg_price       310153 non-null  float64
 8   Planned_delivery_time  240068 non-null  float64
 9   Reorder_point          310160 non-null  float64
 10  Max_Stock_Level        310160 non-null  float64
 11  Storage_Bin            90218 non-null   object 
 12  MRP_Type               175183 non-null  object 
 13  Material_Group         310157 non-null  object 
 14  Purchasing_Group       291806 non-nu

In [14]:
mm["Material_Number"].value_counts

<bound method IndexOpsMixin.value_counts of 0          7803141642
1          7832234005
2         75030202210
3         75030202220
4         75030202260
             ...     
310155    RE377000008
310156    RE377000008
310157    RE377000007
310158    RE377000007
310159    RE377000007
Name: Material_Number, Length: 310160, dtype: object>

In [15]:
mm["Material_Number"].duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
310155     True
310156     True
310157     True
310158     True
310159     True
Name: Material_Number, Length: 310160, dtype: bool

In [16]:
mm["pk_plant_matnum"] = mm["Plant"].astype(str) + "_" + mm["Material_Number"].astype(str)
mm["pk_plant_matnum"].value_counts()

pk_plant_matnum
PO21_PO3315510124          14
PO11_PO3315510121          12
PO21_PO3315510121          12
PO21_PO3315510122          11
OL11_W0OFFICOMPNOTB0003     7
                           ..
LX11_LX32218-1681           1
LX11_LX32218-1702           1
LX11_LX32218-1703           1
LX11_LX32218-1705           1
7811_78107400015            1
Name: count, Length: 293883, dtype: int64

In [17]:
mm.head()

,Material_Type,Material_Number,Material_Description,PlantID,Plant,Plant_Description,UOM,Moving_avg_price,Planned_delivery_time,Reorder_point,...,CompanyCode,CompanyDesc,Consigment_Onhand,Consigment_Blocked,MaterialConsigment,Currency,flagdeleted,createdatetime,updatedatetime,pk_plant_matnum
0,3.0,7803141642,"SW,PROXI,3RG4014-0AG30-Z,SIEMENS",PT014,7811,TPC-Ban Pong,PC,5400.00,NaN,2.0,...,780,"THAI PAPER CO.,LTD.",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:12:55.877,7811_7803141642
1,32.0,7832234005,"FELT,PICK UP PM#4,5,23.00X3.95 M,ATROVIT",PT014,7811,TPC-Ban Pong,PC,314484.73,NaN,0.0,...,780,"THAI PAPER CO.,LTD.",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:28:49.973,7811_7832234005
2,3.0,75030202210,"CONTACTOR,CA 501, 1NC,ABB [5]",PT011,7560,SKIC-Wangsa-la,ST,124.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:31:24.79,7560_75030202210
3,3.0,75030202220,"CONTACTOR,CA 510, 1NO,ABB [5]",PT011,7560,SKIC-Wangsa-la,ST,127.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:53:51.327,7560_75030202220
4,3.0,75030202260,"CONTACTOR,CAL 511,1NO 1NC.,ABB [ZN]",PT011,7560,SKIC-Wangsa-la,ST,250.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 12:02:03.967,7560_75030202260


In [18]:
select_mm_matType = mm[
    mm["Material_Type"]
    .fillna(-1)
    .astype(int)
    .map("{:02d}".format)
    .isin(material_type)
]

In [19]:
select_mm_matType = mm[mm["Material_Type"] == 6.0]
select_mm_matType

,Material_Type,Material_Number,Material_Description,PlantID,Plant,Plant_Description,UOM,Moving_avg_price,Planned_delivery_time,Reorder_point,...,CompanyCode,CompanyDesc,Consigment_Onhand,Consigment_Blocked,MaterialConsigment,Currency,flagdeleted,createdatetime,updatedatetime,pk_plant_matnum
8563,6.0,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)",PT011,7560,SKIC-Wangsa-la,PC,1416.66,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:26:53.72,7560_7506809480106
8564,6.0,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA",PT011,7560,SKIC-Wangsa-la,PC,36975.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:13:13.247,7560_7806055050934
8565,6.0,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,PT011,7560,SKIC-Wangsa-la,PC,1700.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:30:06.927,7560_54060106035573
8566,6.0,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034",PT011,7560,SKIC-Wangsa-la,PC,6140.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:47:51.68,7560_54060140106258
8567,6.0,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028",PT011,7560,SKIC-Wangsa-la,PC,2450.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:12:51.51,7560_54060140113243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,6.0,9A06LEDRYMCONT001,3RF2340-1AA06-Solid-state contactor,PT131,9A11,VISY - Store Plant,EA,10195.10,14.0,1.0,...,3200,Vexcel Pack Company Limited,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:12.773,2025-11-15 04:45:06.807,9A11_9A06LEDRYMCONT001
310004,6.0,9A06LAOTHMVALV033,4551305-Electrovalve 1078/M42S CASTEL,PT131,9A11,VISY - Store Plant,EA,8050000.00,14.0,1.0,...,3200,Vexcel Pack Company Limited,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:13.133,2025-11-15 04:45:06.95,9A11_9A06LAOTHMVALV033
310005,6.0,9A06LAOTHMVALV034,4192102-Thermost expansion VALVE SEH-17,PT131,9A11,VISY - Store Plant,EA,8050000.00,14.0,1.0,...,3200,Vexcel Pack Company Limited,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:13.15,2025-11-15 04:45:06.963,9A11_9A06LAOTHMVALV034
310018,6.0,LX06320-4207,"BUSHING, BRONZE 5"" OD X 13"" L SOLID",PT115,LX12,UPPC-Philippines,IN,1729.11,21.0,14.0,...,6590,"UNITED PULP AND PAPER CO.,INC",0.0,0.0,NaN,PHP,NaN,2025-11-17 04:45:07.987,2025-11-18 04:45:06.973,LX12_LX06320-4207


In [20]:
#unduplicate values
select_mm_matType["Material_Number"].value_counts()[lambda x: x == 1]

Material_Number
LX06340-5235-1        1
LX06340-5298          1
LX06600-0057-1        1
78061428015951        1
LX06340-5246          1
                     ..
OL06SE7802052A        1
OL06SE7802053A        1
OL06SE7802054A        1
OL06SE7803003A        1
25062A62MOTORA1-49    1
Name: count, Length: 43048, dtype: int64

In [21]:
select_mm_matType["Material_Number"].duplicated().sum()

np.int64(17385)

In [22]:
#check duplicate pk
select_mm_matType["pk_plant_matnum"].value_counts()[lambda x: x > 1]

pk_plant_matnum
259A_25069A290126002    2
C611_C60611210013001    2
259A_25069A290126003    2
Name: count, dtype: int64

In [23]:
# Deduplicate select first row
select_mm_matType_no_dup = select_mm_matType.drop_duplicates(subset=["pk_plant_matnum"], keep="first")
select_mm_matType_no_dup["pk_plant_matnum"].value_counts()[lambda x: x > 1]

Series([], Name: count, dtype: int64)

In [24]:
#duplicate values
select_mm_matType_no_dup["Material_Number"].value_counts()[lambda x: x > 1]

Material_Number
78060628955964     7
75060672337117     7
78060626102000     6
78064670512951     6
78067302594982     6
                  ..
LX06373-0031       2
75066834201211     2
75066830007236     2
25068A230140023    2
75066833902133     2
Name: count, Length: 11256, dtype: int64

In [25]:
#unduplicate values
select_mm_matType_no_dup["Material_Number"].value_counts()[lambda x: x == 1]

Material_Number
LX06336-0497          1
LX06445-0004          1
LX06336-0495          1
LX06336-0304          1
LX06336-0388          1
                     ..
OL06SE7802052A        1
OL06SE7802053A        1
OL06SE7802054A        1
OL06SE7803003A        1
25062A62MOTORA1-49    1
Name: count, Length: 43051, dtype: int64

### PO_TEXT

In [26]:

po = pd.read_csv("Vw_POText.csv", encoding="utf-8")

# แสดงข้อมูล 5 แถวแรก
print(po.head())

             Material TypeText  Line                                  Text  \
0  W0WIREWWAYACCE0013       PO     4  -  POWDER COATED IN TEXTURE RAL 7032   
1  W0WIREWWAYACCE0021       PO     4  -  POWDER COATED IN TEXTURE RAL 7032   
2  W0WIREWWAYACCE0029       PO     4  -  POWDER COATED IN TEXTURE RAL 7032   
3  W0WIREWWAYACCE0037       PO     4  -  POWDER COATED IN TEXTURE RAL 7032   
4  W0WIREWWAYACCE0045       PO     4  -  POWDER COATED IN TEXTURE RAL 7032   

   CreateBy           CreateDateTime  rowid  
0         1  2023-08-28 10:37:38.503      1  
1         1  2023-08-28 10:37:38.503      2  
2         1  2023-08-28 10:37:38.503      3  
3         1  2023-08-28 10:37:38.503      4  
4         1  2023-08-28 10:37:38.503      5  


In [27]:
po.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   Material        1048575 non-null  object
 1   TypeText        1048575 non-null  object
 2   Line            1048575 non-null  int64 
 3   Text            1027025 non-null  object
 4   CreateBy        1048575 non-null  int64 
 5   CreateDateTime  1048575 non-null  object
 6   rowid           1048575 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 56.0+ MB


In [28]:
po["Material"].value_counts()[lambda x: x > 1]

Material
W0OFFIPTWPFORM0084    14
W0INVACS355_7_5KW     14
W0SAFEPSAPACCE0035    14
RE06704118912101      14
RE06704151911100      14
                      ..
7542NS01EE00200578     2
7542NS01EE00200579     2
7542NS01EE00200580     2
7542NS01EE00200384     2
W0CHLBCHEMLABXM202     2
Name: count, Length: 152770, dtype: int64

In [29]:
# รวมข้อความตาม Material
po_combine_text = (
    po.sort_values(["Material", "Line"])                # เรียงตาม Material และ Line
      .groupby("Material")                              # จัดกลุ่มตาม Material
      .agg({"Text": lambda x: " ".join(x.astype(str))}) # รวม Text โดยคั่นด้วย space
      .reset_index()                                     # ให้ Material กลับมาเป็น column
)

po_combine_text

,Material,Text
0,210STKR0000018,STK.PE ใสหลังกลาสซีน กาวธรรมดา 180mmx1000m #BW...
1,225107A400004228,CK SP CK SP33131ORANGE nan ORANGE 4228 nan nan
2,2503,"PLC, MFR PART NAME : PLC CPU, MFR PART NO. : 6..."
3,25032A000045001,"FUSE, SHORT NAME : HCR FUSE 24KV/100A, VOLTAG..."
4,25032A000045002,SENSOR S30B-2011DA#1026822 สถานที่ใช้งาน TRANS...
...,...,...
177924,W410WT140100001,"KRAFT, TYPE : LINER, MFR MODEL NO. : WT140, ..."
177925,W410WT160000001,"KRAFT, TYPE : LINER, MFR MODEL NO. : WT160, ..."
177926,W410YL140130001,KRAFT LINER SDY TL140
177927,W431FORMLW00001,แบบฟอร์มกระดาษต่อเนื่อง ใบกำกับภาษี/ใบส่งของ ก...


## map mat num betweem mat_mas & po text

In [30]:
select_mm_matType_join = select_mm_matType_no_dup.merge(
    po_combine_text,
    how="left",
    left_on="Material_Number",
    right_on="Material"
)

In [31]:
select_mm_matType_join

,Material_Type,Material_Number,Material_Description,PlantID,Plant,Plant_Description,UOM,Moving_avg_price,Planned_delivery_time,Reorder_point,...,Consigment_Onhand,Consigment_Blocked,MaterialConsigment,Currency,flagdeleted,createdatetime,updatedatetime,pk_plant_matnum,Material,Text
0,6.0,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)",PT011,7560,SKIC-Wangsa-la,PC,1416.66,NaN,0.0,...,0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:26:53.72,7560_7506809480106,7506809480106,"SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ..."
1,6.0,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA",PT011,7560,SKIC-Wangsa-la,PC,36975.00,NaN,0.0,...,0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:13:13.247,7560_7806055050934,7806055050934,"FLOWMETER, SHORT NAME : FLOWMETER, FLOW RANGE..."
2,6.0,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,PT011,7560,SKIC-Wangsa-la,PC,1700.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:30:06.927,7560_54060106035573,54060106035573,"ACID PHOSPHORIC COMERCIAL GRADE, MACHINE CODE ..."
3,6.0,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034",PT011,7560,SKIC-Wangsa-la,PC,6140.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:47:51.68,7560_54060140106258,54060140106258,"SHORT NAME : BEARING,ACC, MFR NAME : SKF, TY..."
4,6.0,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028",PT011,7560,SKIC-Wangsa-la,PC,2450.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:12:51.51,7560_54060140113243,54060140113243,"BEARING,ACC, MFR NAME : SKF, TYPE : WITHDRAW..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71684,6.0,9A06LEDRYMCONT001,3RF2340-1AA06-Solid-state contactor,PT131,9A11,VISY - Store Plant,EA,10195.10,14.0,1.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:12.773,2025-11-15 04:45:06.807,9A11_9A06LEDRYMCONT001,NaN,NaN
71685,6.0,9A06LAOTHMVALV033,4551305-Electrovalve 1078/M42S CASTEL,PT131,9A11,VISY - Store Plant,EA,8050000.00,14.0,1.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:13.133,2025-11-15 04:45:06.95,9A11_9A06LAOTHMVALV033,NaN,NaN
71686,6.0,9A06LAOTHMVALV034,4192102-Thermost expansion VALVE SEH-17,PT131,9A11,VISY - Store Plant,EA,8050000.00,14.0,1.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:13.15,2025-11-15 04:45:06.963,9A11_9A06LAOTHMVALV034,NaN,NaN
71687,6.0,LX06320-4207,"BUSHING, BRONZE 5"" OD X 13"" L SOLID",PT115,LX12,UPPC-Philippines,IN,1729.11,21.0,14.0,...,0.0,0.0,NaN,PHP,NaN,2025-11-17 04:45:07.987,2025-11-18 04:45:06.973,LX12_LX06320-4207,LX06320-4207,"Bushing, Bronze size: 5"" OD x 13"" Length , Sol..."


In [32]:
select_mm_matType_mapped = select_mm_matType_join[select_mm_matType_join["Material"].notna()]
select_mm_matType_mapped

,Material_Type,Material_Number,Material_Description,PlantID,Plant,Plant_Description,UOM,Moving_avg_price,Planned_delivery_time,Reorder_point,...,Consigment_Onhand,Consigment_Blocked,MaterialConsigment,Currency,flagdeleted,createdatetime,updatedatetime,pk_plant_matnum,Material,Text
0,6.0,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)",PT011,7560,SKIC-Wangsa-la,PC,1416.66,NaN,0.0,...,0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:26:53.72,7560_7506809480106,7506809480106,"SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ..."
1,6.0,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA",PT011,7560,SKIC-Wangsa-la,PC,36975.00,NaN,0.0,...,0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:13:13.247,7560_7806055050934,7806055050934,"FLOWMETER, SHORT NAME : FLOWMETER, FLOW RANGE..."
2,6.0,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,PT011,7560,SKIC-Wangsa-la,PC,1700.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:30:06.927,7560_54060106035573,54060106035573,"ACID PHOSPHORIC COMERCIAL GRADE, MACHINE CODE ..."
3,6.0,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034",PT011,7560,SKIC-Wangsa-la,PC,6140.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:47:51.68,7560_54060140106258,54060140106258,"SHORT NAME : BEARING,ACC, MFR NAME : SKF, TY..."
4,6.0,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028",PT011,7560,SKIC-Wangsa-la,PC,2450.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:12:51.51,7560_54060140113243,54060140113243,"BEARING,ACC, MFR NAME : SKF, TYPE : WITHDRAW..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71672,6.0,78062409570000,"COUPLING,COMPLETE,DODGE,PX70,DODGE(5)",PT011,7560,SKIC-Wangsa-la,ST,9630.00,0.0,0.0,...,0.0,0.0,NaN,THB,NaN,2025-11-11 04:45:07.363,2025-11-13 04:45:08.85,7560_78062409570000,78062409570000,"COUPLING, SHORT NAME : COUPLING, MFR PART NAM..."
71681,6.0,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC",PT016,7813,TPC-Ban Pong,PC,3865.00,0.0,0.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:07.173,2025-11-15 04:45:04.583,7813_78063621820573,78063621820573,"FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."
71682,6.0,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC",PT014,7811,TPC-Ban Pong,PC,3865.00,0.0,0.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:08.417,2025-11-15 04:45:04.69,7811_78063621820573,78063621820573,"FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."
71683,6.0,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC",PT022,K011,SNP-Ban Pong,PC,3865.00,0.0,0.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:08.953,2025-11-15 04:45:05.063,K011_78063621820573,78063621820573,"FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."


In [33]:
select_mm_matType_mapped["pk_plant_matnum"].value_counts()[lambda x: x > 1]

Series([], Name: count, dtype: int64)

In [34]:
select_select_mm_matType = select_mm_matType_mapped[["pk_plant_matnum", "Material_Type", "Plant", "Material_Number", "PlantID", "Plant_Description", "UOM", "Material", "Material_Description", "Text"]]
select_select_mm_matType

,pk_plant_matnum,Material_Type,Plant,Material_Number,PlantID,Plant_Description,UOM,Material,Material_Description,Text
0,7560_7506809480106,6.0,7560,7506809480106,PT011,SKIC-Wangsa-la,PC,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)","SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ..."
1,7560_7806055050934,6.0,7560,7806055050934,PT011,SKIC-Wangsa-la,PC,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA","FLOWMETER, SHORT NAME : FLOWMETER, FLOW RANGE..."
2,7560_54060106035573,6.0,7560,54060106035573,PT011,SKIC-Wangsa-la,PC,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,"ACID PHOSPHORIC COMERCIAL GRADE, MACHINE CODE ..."
3,7560_54060140106258,6.0,7560,54060140106258,PT011,SKIC-Wangsa-la,PC,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034","SHORT NAME : BEARING,ACC, MFR NAME : SKF, TY..."
4,7560_54060140113243,6.0,7560,54060140113243,PT011,SKIC-Wangsa-la,PC,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028","BEARING,ACC, MFR NAME : SKF, TYPE : WITHDRAW..."
...,...,...,...,...,...,...,...,...,...,...
71672,7560_78062409570000,6.0,7560,78062409570000,PT011,SKIC-Wangsa-la,ST,78062409570000,"COUPLING,COMPLETE,DODGE,PX70,DODGE(5)","COUPLING, SHORT NAME : COUPLING, MFR PART NAM..."
71681,7813_78063621820573,6.0,7813,78063621820573,PT016,TPC-Ban Pong,PC,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC","FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."
71682,7811_78063621820573,6.0,7811,78063621820573,PT014,TPC-Ban Pong,PC,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC","FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."
71683,K011_78063621820573,6.0,K011,78063621820573,PT022,SNP-Ban Pong,PC,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC","FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."


In [35]:
print(select_select_mm_matType[["Material_Description","UOM", "Text"]]
      .head()
      .to_string())

                     Material_Description UOM                                                                                                                                                                                                                                                                                                                                                                                                                                                                 Text
0          SPROCKET,CHAIN,49 T ,1/2 "(ZN)  PC                                                                                                          SPROCKET,CHAIN, SHORT NAME  : SPROCKET,CHAIN, NO. OF STRANDS  : 1, NO. OF TEETH  : 49, FOR CHAIN NO.  : ?, PITCH  : 1/2 ", MATERIAL  : CARBON STEEL, APPLICATION  : SILENT CHAIN STOKER PB 8, REMARK  : SPROCKET DRIVEN MAT. MEDIUM CARBON STEEL TOOTH HARD SURFACE NO. OF TEETH 49, PITCH 0.500" AS PER DWG.NO. 106 309 B 001 FOR SILENT CHAIN STOKER PB 8
1 

In [36]:
select_select_mm_matType["UOM"].value_counts().count()

np.int64(40)

In [37]:
select_select_mm_matType.to_csv("select_select_mm_matType_06.csv", index=False, encoding="utf-8-sig")

## 0. Combine field

### 0.1 for raw_combine_for_embedding (without uom)

In [38]:
select_select_mm_matType["raw_combine_for_embedding"] = "short_description: " + select_select_mm_matType["Material_Description"] + " long_description: " + select_select_mm_matType["Text"]
select_select_mm_matType.head(1)

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_7444/721366568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_select_mm_matType["raw_combine_for_embedding"] = "short_description: " + select_select_mm_matType["Material_Description"] + " long_description: " + select_select_mm_matType["Text"]


,pk_plant_matnum,Material_Type,Plant,Material_Number,PlantID,Plant_Description,UOM,Material,Material_Description,Text,raw_combine_for_embedding
0,7560_7506809480106,6.0,7560,7506809480106,PT011,SKIC-Wangsa-la,PC,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)","SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ...","short_description: SPROCKET,CHAIN,49 T ,1/2 ""(..."


### 0.2 for raw_combine_for_keyword (add uom)

In [39]:
select_select_mm_matType["raw_combine_for_keyword"] = select_select_mm_matType["Material_Description"] + " " + select_select_mm_matType["Text"] + " " + select_select_mm_matType["UOM"]
select_select_mm_matType.head(1)

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_7444/1177480277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_select_mm_matType["raw_combine_for_keyword"] = select_select_mm_matType["Material_Description"] + " " + select_select_mm_matType["Text"] + " " + select_select_mm_matType["UOM"]


,pk_plant_matnum,Material_Type,Plant,Material_Number,PlantID,Plant_Description,UOM,Material,Material_Description,Text,raw_combine_for_embedding,raw_combine_for_keyword
0,7560_7506809480106,6.0,7560,7506809480106,PT011,SKIC-Wangsa-la,PC,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)","SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ...","short_description: SPROCKET,CHAIN,49 T ,1/2 ""(...","SPROCKET,CHAIN,49 T ,1/2 ""(ZN) SPROCKET,CHAIN,..."


## 1. Data Cleaning & Normalization
สรุป: ทำให้ข้อมูลสะอาด สม่ำเสมอ พร้อมให้ embedding และ BM25 ใช้ได้ดี
- Uppercase ทั้งหมด
- Normalize whitespace
- เก็บ character สำคัญของ part code (- / _ . ( ))
- Remove noise tokens (EA, PCS, ITEM, DESC)
- รวม material_description + PO_text → context

In [40]:
def normalize_whitespace(text: str) -> str:
    # แทน \n \t ด้วย space และลด space ซ้อน
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def remove_duplicate_commas_periods(text: str) -> str:
    # ลด ,, → , และ .. → .
    text = re.sub(r",{2,}", ",", text)
    text = re.sub(r"\.{2,}", ".", text)
    return text

def lowercase_text(text: str) -> str:
    return text.lower()


# --------- Main Cleaning Function ---------
def cleaning_features(text: str) -> str:
    if pd.isna(text):
        return ""

    # 1. normalize whitespace
    text = normalize_whitespace(text)

    # 2. remove duplicate punctuation
    text = remove_duplicate_commas_periods(text)

    # 3. keep punctuation that conveys meaning → (ไม่มีการลบใด ๆ)

    # 4. lowercase for embedding stability
    # text = lowercase_text(text)

    # 5. Normalize dash/underscore variants
    text = normalize_dashes(text)

    return text.strip()

def normalize_dashes(text):
    return text.replace("–", "-").replace("—", "-")

def extract_partcodes(text):
    """Extract part codes/model numbers from text"""
    # Pattern 1: Alphanumeric with separators (e.g., 3310-ATN9, NU-318)
    part_pattern = r"(?=[A-Z0-9]*[A-Z])[A-Z0-9]+(?:[-_/\.][A-Z0-9]+)+"
    parts = re.findall(part_pattern, text.upper())
    
    # Pattern 2: Model numbers (e.g., 3310, 7306)
    model_pattern = r'\b[A-Z0-9]{3,}\b'
    models = re.findall(model_pattern, text.upper())
    
    # Combine and deduplicate
    all_codes = parts + models
    return list(dict.fromkeys(all_codes))

# --------- Build Embedding Text ---------
def build_raw_combine_for_embedding(row):
    sd = cleaning_features(row["Material_Description"]) if pd.notna(row["Material_Description"]) else ""
    ld = cleaning_features(row["Text"]) if pd.notna(row["Text"]) else ""

    combined = f"short_description: {sd} long_description: {ld}"

    # extract part-code to reinforce semantic
    parts = extract_partcodes(combined)
    if parts:
        combined += " part_codes: " + ", ".join(parts)

    # lowercase everything for embedding stability
    combined = lowercase_text(combined)

    return combined.strip()

### 1.1 cleansing for sementic search (embed)
- 1. normalize whitespace
- 2. remove duplicate punctuation
- 3. Normalize dash/underscore variants
- 4. extract part-code to reinforce semantic
- 5. lowercase everything for embedding stability

In [41]:
select_select_mm_matType["clean_combine_for_embedding"] = (
    select_select_mm_matType.apply(build_raw_combine_for_embedding, axis=1)
)

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_7444/420435984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_select_mm_matType["clean_combine_for_embedding"] = (


In [42]:
with pd.option_context('display.max_colwidth', None):
    print(
        select_select_mm_matType["clean_combine_for_embedding"]
            .head(5)
            .to_string()
    )

0                                                                                                                                                                                                                                                                                                                                                                       short_description: sprocket,chain,49 t ,1/2 "(zn) long_description: sprocket,chain, short name : sprocket,chain, no. of strands : 1, no. of teeth : 49, for chain no. : ?, pitch : 1/2 ", material : carbon steel, application : silent chain stoker pb 8, remark : sprocket driven mat. medium carbon steel tooth hard surface no. of teeth 49, pitch 0.500" as per dwg.no. 106 309 b 001 for silent chain stoker pb 8 part_codes: short_description, long_description, dwg.no, sprocket, chain, short, name, strands, teeth, for, pitch, material, carbon, steel, application, silent, stoker, remark, driven, mat, medium, tooth, hard, surface, 500, per, dw

### 1.2 cleansing for keyword search

In [43]:
with pd.option_context('display.max_colwidth', None):
    print(
        select_select_mm_matType["raw_combine_for_keyword"]
            .head(20)
            .to_string()
    )

0                                                                                                                                                                   SPROCKET,CHAIN,49 T ,1/2 "(ZN) SPROCKET,CHAIN, SHORT NAME  : SPROCKET,CHAIN, NO. OF STRANDS  : 1, NO. OF TEETH  : 49, FOR CHAIN NO.  : ?, PITCH  : 1/2 ", MATERIAL  : CARBON STEEL, APPLICATION  : SILENT CHAIN STOKER PB 8, REMARK  : SPROCKET DRIVEN MAT. MEDIUM CARBON STEEL TOOTH HARD SURFACE NO. OF TEETH 49, PITCH 0.500" AS PER DWG.NO. 106 309 B 001 FOR SILENT CHAIN STOKER PB 8 PC
1                                                                                                                                                                                                                                                                                                                                                                   FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA FLOWMETER, SHORT NAME  : FLOWMETER, FLOW RANGE  : 300L/MIN, MFR

In [44]:
# ยังติด no. ใน keyword_list

In [45]:
# Function before extracting identifier_keyword
def clean_quotes(text: str) -> str:
    """
    - remove "word" ที่ไม่มีเลขด้านใน
    - remove stray quotes เช่น "zn → zn
    - แปลง inch: 3/4" → 3/4 inch
    - (สุดท้ายค่อย normalize spacing)
    """

    # ---------------------------------------------------------
    # A) remove quoted words without digits → remove entirely
    # "SKF" → space
    # "MAHLE" → space
    # "abc-def" → space
    # ---------------------------------------------------------
    def remove_non_numeric_quotes(m):
        inside = m.group(1)
        if not re.search(r'\d', inside):
            return " "     # ไม่มีเลข → remove
        return m.group(0)  # มีเลข → keep

    text = re.sub(r'"([^"]*)"', remove_non_numeric_quotes, text)

    # ---------------------------------------------------------
    # B) remove stray quotes (quote เดี่ยว ๆ ที่เหลือ)
    # เช่น "zn → zn,  zn" → zn
    # ---------------------------------------------------------
    text = re.sub(r'(^| )"(\w+)', r'\1\2', text)
    text = re.sub(r'(\w+)"( |$)', r'\1\2', text)

    # ---------------------------------------------------------
    # C) convert inch (only numeric + double-quote)
    # ทำหลังจาก clean quotes แล้ว
    # ---------------------------------------------------------
    text = re.sub(r'(\d+/\d+|\d+(?:\.\d+)?)"', r'\1 inch', text)
    text = re.sub(r'(\d+/\d+|\d+(?:\.\d+)?)\s*"(\s|$)', r'\1 inch ', text)

    # ---------------------------------------------------------
    # D) normalize spaces
    # ---------------------------------------------------------
    text = re.sub(r'\s+', ' ', text).strip()

    return text

special_chars = ["?"]

def clean_special_chars(text: str, special_chars: list) -> str:
    """
    Remove / replace special characters defined in special_chars list.
    Replace with space to preserve token boundaries.
    """
    if not text:
        return text

    # escape chars for regex safety (เช่น ?, *, ., etc.)
    escaped = [re.escape(c) for c in special_chars]
    pattern = f"[{''.join(escaped)}]"

    # replace with space
    text = re.sub(pattern, " ", text)

    # collapse multiple spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text


def normalize_spacing(text):
    """
    - ใส่ space ก่อน–หลัง comma
    - compress multiple spaces
    """

    # Add space around commas
    text = re.sub(r'\s*,\s*', ' , ', text)

    # collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def normalize_uom(text: str) -> str:

    # Convert number + optional space + " → number inch
    # Examples:
    #   1/2"  → 1/2 inch
    #   1/2 " → 1/2 inch
    #   10"   → 10 inch
    #   10 "  → 10 inch
    text = re.sub(r'\b(\d+(?:/\d+)?)(\s*)"(?![a-zA-Z])', r'\1 inch', text)

    # --- 2. millimeter → mm ---
    text = re.sub(r'\bmil{0,1}imet(er|re|ers|res)\b', 'mm', text)

    # --- 3. kg variations → kg ---
    text = re.sub(r'\bkg\.?\b', 'kg', text)
    text = re.sub(r'\bkgs?\.?\b', 'kg', text)
    text = re.sub(r'\bkilogram(s)?\b', 'kg', text)

    # --- 4. bars → bar ---
    text = re.sub(r'\bbars\b', 'bar', text)

    # --- 5. centimeter → cm ---
    text = re.sub(r'\bcentimet(er|re|ers|res)\b', 'cm', text)
    text = re.sub(r'\bcentimeter(s)?\b', 'cm', text)

    # --- 6. meter → m ---  
    # (ไม่แตะกรณี M560475D1 เพราะต้องเป็น word boundary)
    text = re.sub(r'\bmet(er|re|ers|res)\b', 'm', text)
    text = re.sub(r'\bmeters\b', 'm', text)

    # --- 7. liter → L ---
    text = re.sub(r'\blitre(s)?\b', 'L', text)
    text = re.sub(r'\bliter(s)?\b', 'L', text)

    # --- 8. ton / tonne ---
    text = re.sub(r'\btonne(s)?\b', 'ton', text)
    text = re.sub(r'\btons\b', 'ton', text)

    return text

# ============================================================
# 2) Preprocess text (normalize punctuation / spacing / dash)
# ============================================================
def normalize_text(t: str) -> str:
    t = t.lower()

    # normalize dash variations
    t = t.replace("–", "-").replace("—", "-").replace("_", "-")

    # remove duplicate punctuations (,,, ---> ,)
    # reduce duplicate punctuation (safe)
    t = re.sub(r'([!?,.;]{2,})', lambda m: m.group()[0], t)

    # # remove unwanted punctuations: * and :
    t = t.replace("*", " ")
    t = t.replace(":", " ")

    # # normalize spaces
    t = re.sub(r"\s+", " ", t).strip()

    return t

def normalize_temperature(text):
    # -------- Celsius --------
    # ครอบคลุม:
    # DEGREE CELSIUS, DEG CELSISU, DEG C., 
    #  C, DEG.CELSIUS,
    # DEGREECELSIUS, DEGREE OF CELSIUS, DEGREE OF C, DEG C.
    celsius_patterns = [
        r'\b(\d+)\s*(deg\.?|degree\.?|degrees\.?)\s*(of\s*)?(celsius|celcius|celsisu|cels|cel|c)\b',
        r'\b(\d+)\s*(°)\s*c\b',
        r'\b(\d+)\s*c\b'
    ]

    for p in celsius_patterns:
        text = re.sub(p, r'\1°C', text, flags=re.I)

    # -------- Fahrenheit --------
    fahrenheit_patterns = [
        r'\b(\d+)\s*(deg\.?|degree\.?|degrees\.?)\s*(of\s*)?(fahrenheit|farenheit|farenhiet|f)\b',
        r'\b(\d+)\s*(°)\s*f\b',
        r'\b(\d+)\s*f\b'
    ]

    for p in fahrenheit_patterns:
        text = re.sub(p, r'\1°F', text, flags=re.I)

    return text

def normalize_technical_terms(text: str) -> str:
    # ------------------------------------------------------------
    # STEP 1) handle dia. followed by number → insert space only
    # ------------------------------------------------------------
    text = re.sub(r'\bdia\.(?=\d)', r'dia. ', text, flags=re.I)

    # ------------------------------------------------------------
    # STEP 2) normalize exact tokens only → dia.
    #   - diameter
    #   - diam.
    #   - dia
    #   - dia.  (เฉพาะกรณีเป็น token เดี่ยว ๆ)
    #   - ø φ Ø Φ
    # ------------------------------------------------------------
    # normalize diameter
    text = re.sub(r'\bdiameter\b', 'dia.', text, flags=re.I)

    # normalize diam.
    text = re.sub(r'\bdiam\.\b', 'dia.', text, flags=re.I)

    # normalize symbol forms
    text = re.sub(r'[ØøφΦ⌀]', 'dia.', text)

    # normalize dia. (แต่ต้องเป็น token เดี่ยว ไม่ติดตัวเลข)
    text = re.sub(r'\bdia\.(?!\d)', 'dia.', text, flags=re.I)

    # ------------------------------------------------------------
    # STEP 3) collapse spaces
    # ------------------------------------------------------------
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [46]:
# Temp for extract_identifier_candidates
# ============================================================
# 1) Utility regex
# ============================================================

# แนวๆ CODE/PART NUMBER:
code_patterns = [
    r"[A-Z]{1,4}\d{1,6}[A-Z0-9\-]*",    # ABC1234 / E70340M-K059521
    r"\d{2,4}[A-Z]{1,4}\d{2,6}",        # 512ABC1234
    r"[A-Z]{1,4}\d{2,4}",               # TLFH-4CA (ก่อน normalize)
    r"\d{2,4}[A-Z]{1,4}",               
    # r"\bAH[- ]?\d{3,5}\b",              # AH24034
]

code_regex = re.compile("|".join(code_patterns), re.IGNORECASE)
# ============================================================
# 4) Extract CODE patterns for exact-match boosting
# ============================================================
def extract_codes(t: str):
    return list(set(code_regex.findall(t)))

In [47]:
# Extract identifier keyword
# 1. extract_identifier_candidates

IDENTIFIER_PATTERNS = [

    # 1) Classic material / part code
    r"\b[A-Z]{1,4}[- ]?\d{2,6}[A-Z0-9]*\b",

    # 2) Complex code with separators
    r"\b[A-Z0-9]{2,}[-/][A-Z0-9-/]{2,}\b",

    # 3) Drawing / model no.
    r"\b[A-Z]{1,3}\d{4,}[A-Z0-9/.-]*\b",

    # 4) Metric thread / fastener
    r"\bM\d{2,3}\s*[xX]\s*L?\d{2,4}\b",

    # 5) Dimension compound
    r"\b\d{2,4}\s*[xX]\s*\d{2,5}\s*(mm|cm|m)?\b",

    # 6) Pipe / standard
    r"\bDN\d{2,4}\b",
]

IDENTIFIER_REGEX = re.compile("|".join(IDENTIFIER_PATTERNS), re.IGNORECASE)


def extract_identifier_candidates(text: str) -> list[str]:
    if not isinstance(text, str):
        return []
    return list(set(m.group(0) for m in IDENTIFIER_REGEX.finditer(text)))


# 2. normalize_identifier
def normalize_identifier(code: str) -> str:
    c = code.strip().upper()

    # 1) normalize spaces around separators
    c = re.sub(r"\s*-\s*", "-", c)
    c = re.sub(r"\s*/\s*", "/", c)
    c = re.sub(r"\s*[xX]\s*", "x", c)

    # 2) AH 24034 → AH-24034
    c = re.sub(r"\b([A-Z]{1,4})\s+(\d{2,6})\b", r"\1-\2", c)

    # 3) normalize unit casing
    c = re.sub(r"\bMM\b", "MM", c)

    return c

# 3. filter_identifier
def is_valid_identifier(code: str) -> bool:

    # too short
    if len(code) < 4:
        return False

    # pure number
    if code.isdigit():
        return False

    # single dimension like 10MM
    if re.fullmatch(r"\d{1,4}(MM|CM|M)?", code):
        return False

    # must contain at least one digit + one letter
    if not (re.search(r"[A-Z]", code) and re.search(r"\d", code)):
        return False

    return True

def extract_identifier_keywords(text: str) -> list[str]:
    raw_candidates = extract_identifier_candidates(text)

    normalized = [normalize_identifier(c) for c in raw_candidates]

    filtered = [c for c in normalized if is_valid_identifier(c)]

    return sorted(set(filtered))



In [48]:
def remove_no(text):
    """
    remove only stopword: "no."
    """
    tokens = text.split()
    tokens = [t for t in tokens if t.lower() not in ["no."]]
    return " ".join(tokens)

# ============================================================
# 5) Remove stopwords for BM25 keyword clarity
# ============================================================
# def remove_stopwords(t: str) -> str:
#     tokens = t.split()
#     tokens = [tok for tok in tokens if tok not in stopwords]
#     return " ".join(tokens)
stopwords_list = ["short",
    "name",
    "long",
    "description",
    "remark",
    "application",
    "equipment",
    "machine",
    "mfr",
    "maker",
    "model",
    "type",
    "serial",
    "supplier",
    "shop",
    "drawing",
    "dwg.",
    "dwg",
    "no.",
    "number",
    "offer",
    "item",
    "position",
    "use for",
    "used for"
    "used",
    "?",
    "from",
    "part",
    "partno",
    "code",
    "unit",
    "qty.",
    "qty",
    "size",
    "pressure",
    "range"
]

stopwords = set(stopwords_list)

def remove_stopwords(t: str) -> str:
    tokens = t.split()
    cleaned = []
    for tok in tokens:
        tok_clean = tok.strip(",:;")  # ลบ punctuation รอบ ๆ
        if tok_clean and tok_clean not in stopwords:  # <--- ป้องกัน "" 
            cleaned.append(tok_clean)
    return " ".join(cleaned)

# ============================================================
# 6) Create keyword_list field
# ============================================================
def build_keyword_list(clean_text: str, codes: list):
    tokens = clean_text.split()
    return list(sorted(set(tokens + codes)))

In [49]:
# ============================================================
# Main logic for keyword pipeline
# ============================================================
def clean_keyword_pipeline(text: str):
    if not isinstance(text, str):
        return "", [], [], []

    t = text

    # 1) special " cleaner (change " to inch / garbage / remove)
    t = clean_quotes(t)

    # 2) clean special chars
    t = clean_special_chars(t, special_chars)

    # 3) Add space around comma
    t = normalize_spacing(t)

    # # 4) normalize
    t = normalize_text(t)

    # # 5) normalize UOM
    t = normalize_uom(t)

    # # 6) normalize degree celsius and Fahrenheit
    t = normalize_temperature(t)

    # # 7) normalize technical term
    t = normalize_technical_terms(t)

    # # 8) extract codes
    old_codes = extract_codes(t)
    codes = extract_identifier_keywords(t)

    # # 9) remove no.
    t = remove_no(t)

    # 10) remove stopwords
    t_no_stop = remove_stopwords(t)

    # 5) build keyword list
    keyword_list = build_keyword_list(t_no_stop, codes)
    # codes = ""
    # keyword_list = ""

    return t, codes, old_codes, keyword_list



select_select_mm_matType["clean_combine_for_keyword"], select_select_mm_matType["keyword_codes"], select_select_mm_matType["keyword_old_codes"], select_select_mm_matType["keyword_list"] = zip(
    *select_select_mm_matType["raw_combine_for_keyword"].map(clean_keyword_pipeline)
)

select_select_mm_matType.head()

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_7444/2088084179.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_select_mm_matType["clean_combine_for_keyword"], select_select_mm_matType["keyword_codes"], select_select_mm_matType["keyword_old_codes"], select_select_mm_matType["keyword_list"] = zip(
/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_7444/2088084179.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_select_mm_matType["clean_combine_for_keyword"], select_s

,pk_plant_matnum,Material_Type,Plant,Material_Number,PlantID,Plant_Description,UOM,Material,Material_Description,Text,raw_combine_for_embedding,raw_combine_for_keyword,clean_combine_for_embedding,clean_combine_for_keyword,keyword_codes,keyword_old_codes,keyword_list
0,7560_7506809480106,6.0,7560,7506809480106,PT011,SKIC-Wangsa-la,PC,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)","SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ...","short_description: SPROCKET,CHAIN,49 T ,1/2 ""(...","SPROCKET,CHAIN,49 T ,1/2 ""(ZN) SPROCKET,CHAIN,...","short_description: sprocket,chain,49 t ,1/2 ""(...","sprocket , chain , 49 t , 1/2 inch(zn) sprocke...",[B-001],[],"[0.500, 001, 1, 1/2, 106, 309, 49, 8, B-001, a..."
1,7560_7806055050934,6.0,7560,7806055050934,PT011,SKIC-Wangsa-la,PC,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA","FLOWMETER, SHORT NAME : FLOWMETER, FLOW RANGE...","short_description: FLOWMETER,300L/MIN,TLFH-4CA...","FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA FLOWMETER, ...","short_description: flowmeter,300l/min,tlfh-4ca...","flowmeter , 300l/min , tlfh-4ca , kytola flowm...","[300L/MIN, TLFH-4CA]",[300l],"[300L/MIN, 300l/min, TLFH-4CA, flow, flowmeter..."
2,7560_54060106035573,6.0,7560,54060106035573,PT011,SKIC-Wangsa-la,PC,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,"ACID PHOSPHORIC COMERCIAL GRADE, MACHINE CODE ...",short_description: ACID PHOSPHORIC COMERCIAL G...,ACID PHOSPHORIC COMERCIAL GRADE ACID PHOSPHORI...,short_description: acid phosphoric comercial g...,acid phosphoric comercial grade acid phosphori...,"[E70340M, E70540M, K059001, K059088, K059249, ...","[e70540m-l014003, e70540m-l014001, e70340m-k05...","[(กรดบัดกรี), 35, 512, E70340M, E70540M, K0590..."
3,7560_54060140106258,6.0,7560,54060140106258,PT011,SKIC-Wangsa-la,PC,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034","SHORT NAME : BEARING,ACC, MFR NAME : SKF, TY...","short_description: BEARING,ACC,WITHDRAWAL SLEE...","BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034 SHORT N...","short_description: bearing,acc,withdrawal slee...","bearing , acc , withdrawal sleeve , ah 24034 s...","[AH-24034, M-180]",[],"[180, 24034, 3, AH-24034, M-180, acc, adapter,..."
4,7560_54060140113243,6.0,7560,54060140113243,PT011,SKIC-Wangsa-la,PC,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028","BEARING,ACC, MFR NAME : SKF, TYPE : WITHDRAW...","short_description: BEARING,ACC,WITHDRAWAL SLEE...","BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028 BEARING...","short_description: bearing,acc,withdrawal slee...","bearing , acc , withdrawal sleeve , ah 24028 b...",[AH-24028],[],"[2, 24028, AH-24028, acc, adapter, ah, bearing..."


In [50]:
with pd.option_context('display.max_colwidth', None):
    print(
        select_select_mm_matType[["pk_plant_matnum","raw_combine_for_keyword","clean_combine_for_keyword","keyword_codes","keyword_old_codes","keyword_list"]]
            .head(30)
            .to_string()
    )

        pk_plant_matnum                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          raw_combine_for_keyword                                                                                                                                                                                                                                                                                                                                                                                                

In [51]:
with pd.option_context('display.max_colwidth', None):
    print(
        select_select_mm_matType
            .head(10)
            .to_string()
    )

       pk_plant_matnum  Material_Type Plant Material_Number PlantID Plant_Description UOM        Material                    Material_Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Text                                                                                                                                                                                                                                                                                                                                                                

In [52]:
select_select_mm_matType[
    ["raw_combine_for_keyword", "clean_combine_for_keyword", "keyword_codes", "keyword_list"]
].head(30).to_csv(
    "check_30_new1.csv",
    index=False,
    encoding="utf-8-sig"
)

In [53]:
edges = pd.read_csv("edges.csv")
sample50 = edges.sample(n=50, random_state=42)  # 50 rows = 100 unique PKs
sample50.head()

,MODEL_BRAND,Key_mat_1,Plant_1,Mat_1,Key_mat_2,Plant_2,Mat_2
244,6207_SKF,7560_75060624107999,7560,75060624107999,PO11_PO06131330639,PO11,PO06131330639
101,22314 E/C3_SKF,7560_75060673914245,7560,75060673914245,PO21_PO06131335343,PO21,PO06131335343
1080,22313 EK/C3_SKF,7511_75060672353414,7511,75060672353414,PO21_PO06131335101,PO21,PO06131335101
583,6020_SKF,7812_78060620220561,7812,78060620220561,7813_78060617525257,7813,78060617525257
752,6219/C3_SKF,7560_75060624419300,7560,75060624419300,PO11_PO06131330853,PO11,PO06131330853


In [54]:
# สร้าง set ของ key เพื่อให้เช็กได้เร็ว
valid_keys = set(select_select_mm_matType["pk_plant_matnum"])

# สร้าง mask ว่า record ไหนผ่านเงื่อนไข (ต้องมีทั้งคู่)
mask = (
    sample50["Key_mat_1"].isin(valid_keys) &
    sample50["Key_mat_2"].isin(valid_keys)
)

# นับจำนวน record ที่ผ่าน
count_valid_records = mask.sum()

print("จำนวน record ที่มี Key_mat_1 และ Key_mat_2 ครบทั้งคู่:", count_valid_records)


จำนวน record ที่มี Key_mat_1 และ Key_mat_2 ครบทั้งคู่: 43


In [55]:
valid_records = sample50[mask]
valid_records


,MODEL_BRAND,Key_mat_1,Plant_1,Mat_1,Key_mat_2,Plant_2,Mat_2
244,6207_SKF,7560_75060624107999,7560,75060624107999,PO11_PO06131330639,PO11,PO06131330639
583,6020_SKF,7812_78060620220561,7812,78060620220561,7813_78060617525257,7813,78060617525257
752,6219/C3_SKF,7560_75060624419300,7560,75060624419300,PO11_PO06131330853,PO11,PO06131330853
715,6318_SKF,PO11_PO06131330688,PO11,PO06131330688,7511_75060621642160,7511,75060621642160
712,6315_SKF,7811_78060626315000,7811,78060626315000,PO11_PO06131330687,PO11,PO06131330687
174,8V-1700_OPTIBELT,7560_75060766170247,7560,75060766170247,7511_75060755481468,7511,75060755481468
367,22213 EK_SKF,7813_78060670873000,7813,78060670873000,PO11_PO06131335013,PO11,PO06131335013
554,SNL 520-617_SKF,7812_78060656520582,7812,78060656520582,PO21_PO06134761118,PO21,PO06134761118
49,6210-2Z_SKF,PO11_PO06131330645,PO11,PO06131330645,7580_75060626210937,7580,75060626210937
1034,6301_SKF,7560_75060625501999,7560,75060625501999,PO21_PO06131330664,PO21,PO06131330664


In [56]:
# ---------------------------
# 1) เตรียม key 100 ค่า
# ---------------------------
keys_100 = pd.concat([
    sample50["Key_mat_1"],
    sample50["Key_mat_2"]
]).dropna().unique()

print(f"จำนวน unique keys: {len(keys_100)}")

# ---------------------------
# 2) ดึง record จาก table หลัก
# ---------------------------
df_answers = select_select_mm_matType[
    select_select_mm_matType["pk_plant_matnum"].isin(keys_100)
]

print(f"จำนวน record จาก key_mat (answers): {len(df_answers)}")

# ---------------------------
# 3) ตัด record เหล่านี้ออกจาก pool
# ---------------------------
remaining_pool = select_select_mm_matType[
    ~select_select_mm_matType["pk_plant_matnum"].isin(df_answers["pk_plant_matnum"])
]

# ---------------------------
# 4) สุ่มเพิ่ม 200 record (lock seed)
# ---------------------------
df_200 = remaining_pool.sample(
    n=200,
    random_state=42   # 🔒 lock seed
)

print(f"จำนวน random records: {len(df_200)}")

# ---------------------------
# 5) รวมเป็น sample size 300
# ---------------------------

sample_300 = pd.concat([df_answers, df_200], ignore_index=True)

print(f"จำนวน sample ทั้งหมด: {len(sample_300)}")

จำนวน unique keys: 99
จำนวน record จาก key_mat (answers): 92
จำนวน random records: 200
จำนวน sample ทั้งหมด: 292


In [57]:
sample_300["pk_plant_matnum"].is_unique

True

In [58]:
sample_300[['pk_plant_matnum']] == '7511_75060672244000'

,pk_plant_matnum
0,False
1,False
2,False
3,False
4,False
...,...
287,False
288,False
289,False
290,False


## 🚀 Complete Hybrid Search Implementation

**Database**: `apm` | **Schema**: `apm_dev` | **Table**: `spare_part_test`

รวม logic:
1. ✅ Create table with all fields
2. ✅ Insert data + BGE-M3 embeddings
3. ✅ Hybrid search query with weighted scoring

In [8]:
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "black_kalareemhotmail.com",
    "password": "",
    "options": "-c search_path=apm_dev"  # Set default schema
}

In [61]:
import psycopg2
import psycopg2.extras
from pgvector.psycopg2 import register_vector
import numpy as np
from typing import List, Dict, Any, Optional, Tuple
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm

# ============================================================
# Configuration
# ============================================================
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "black_kalareemhotmail.com",
    "password": "",
    "options": "-c search_path=apm_dev,public -c client_encoding=UTF8"  # Set default schema and UTF-8 encoding
}

TABLE_NAME = "spare_part_test"
SCHEMA_NAME = "apm_dev"
EMBEDDING_DIM = 1024  # bge-m3 dimension

# ============================================================
# Database Connection
# ============================================================
def get_connection():
    """Get database connection with pgvector support"""
    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_client_encoding('UTF8')  # Ensure UTF-8 encoding for Thai text
    register_vector(conn)
    return conn

# ============================================================
# 1. CREATE TABLE
# ============================================================
def create_spare_part_table(drop_if_exists: bool = False):
    """
    Create spare_part_test table with all required fields
    
    Args:
        drop_if_exists: If True, drop existing table before creating
    """
    
    conn = get_connection()
    cur = conn.cursor()
    
    try:
        # Drop table if requested
        if drop_if_exists:
            print(f"⚠️  Dropping existing table {SCHEMA_NAME}.{TABLE_NAME}...")
            cur.execute(f"DROP TABLE IF EXISTS {SCHEMA_NAME}.{TABLE_NAME} CASCADE")
        
        # Create table
        create_sql = f"""
        CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
            -- Primary Key
            pk_plant_matnum TEXT PRIMARY KEY,
            
            -- Basic Info
            material_type TEXT,
            plant TEXT,
            material_number TEXT,
            plant_id TEXT,
            plant_description TEXT,
            uom TEXT,
            
            -- Raw Description Fields
            material_description TEXT,
            po_text TEXT,
            
            -- ============================================================
            -- SEMANTIC SEARCH Fields
            -- ============================================================
            raw_combine_for_embedding TEXT,
            clean_combine_for_embedding TEXT,
            embedding vector({EMBEDDING_DIM}),
            
            -- ============================================================
            -- KEYWORD SEARCH Fields (BM25)
            -- ============================================================
            raw_combine_for_keyword TEXT,
            clean_combine_for_keyword TEXT,
            tsv tsvector,
            
            -- Extracted Keywords/Codes
            keyword_codes TEXT[],
            keyword_old_codes TEXT[],
            keyword_list TEXT[],
            
            -- Metadata
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        
        print(f"📦 Creating table {SCHEMA_NAME}.{TABLE_NAME}...")
        cur.execute(create_sql)
        
        # Enable required extensions
        print("🔧 Ensuring extensions are enabled...")
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
        cur.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm")
        
        # Create indexes
        print("🔍 Creating indexes...")
        
        # A) Vector index for semantic search
        cur.execute(f"""
            DO $$
            BEGIN
                IF NOT EXISTS (
                    SELECT 1 FROM pg_indexes 
                    WHERE schemaname='{SCHEMA_NAME}' 
                    AND tablename='{TABLE_NAME}' 
                    AND indexname='idx_{TABLE_NAME}_embedding'
                ) THEN
                    CREATE INDEX idx_{TABLE_NAME}_embedding 
                    ON {SCHEMA_NAME}.{TABLE_NAME} 
                    USING ivfflat (embedding vector_cosine_ops)
                    WITH (lists = 100);
                END IF;
            END
            $$;
        """)
        
        # B) Full-text search index
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_tsv 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin(tsv);
        """)
        
        # C) Trigram index
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_pk_trgm 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (pk_plant_matnum gin_trgm_ops);
        """)
        
        # D) Array indexes for keyword matching
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_keyword_codes 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (keyword_codes);
            
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_keyword_list 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (keyword_list);
        """)
        
        # E) Basic indexes
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_material_type 
            ON {SCHEMA_NAME}.{TABLE_NAME} (material_type);
            
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_uom 
            ON {SCHEMA_NAME}.{TABLE_NAME} (uom);
        """)
        
        # Create trigger for auto-updating tsv
        print("⚙️  Creating trigger for tsvector auto-update...")
        cur.execute(f"""
            CREATE OR REPLACE FUNCTION {SCHEMA_NAME}.{TABLE_NAME}_tsv_trigger() 
            RETURNS TRIGGER AS $$
            BEGIN
                NEW.tsv := to_tsvector('simple', COALESCE(NEW.clean_combine_for_keyword, ''));
                NEW.updated_at := CURRENT_TIMESTAMP;
                RETURN NEW;
            END;
            $$ LANGUAGE plpgsql;
            
            DROP TRIGGER IF EXISTS tsv_update_trigger ON {SCHEMA_NAME}.{TABLE_NAME};
            
            CREATE TRIGGER tsv_update_trigger 
            BEFORE INSERT OR UPDATE ON {SCHEMA_NAME}.{TABLE_NAME}
            FOR EACH ROW 
            EXECUTE FUNCTION {SCHEMA_NAME}.{TABLE_NAME}_tsv_trigger();
        """)
        
        conn.commit()
        print(f"✅ Table {SCHEMA_NAME}.{TABLE_NAME} created successfully!")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Error creating table: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Test
print("="*60)
print("TEST: Create Table")
print("="*60)
# get_connection()
# create_spare_part_table(drop_if_exists=True)  # Uncomment to run

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TEST: Create Table


In [62]:
# ============================================================
# 2. EMBEDDER CLASS (BGE-M3)
# ============================================================
class BGE_M3_Embedder:
    """
    BGE-M3 Embedding Model Wrapper
    """
    def __init__(self, model_name: str = "BAAI/bge-m3", device: str = None):
        """
        Initialize BGE-M3 model
        
        Args:
            model_name: Model identifier (default: BAAI/bge-m3)
            device: 'cuda', 'cpu', or None (auto-detect)
        """
        print(f"🔄 Loading embedding model: {model_name}...")
        self.model = SentenceTransformer(model_name, device=device)
        self.dim = 1024  # bge-m3 output dimension
        print(f"✅ Model loaded! Dimension: {self.dim}")
    
    def embed_texts(
        self, 
        texts: List[str], 
        batch_size: int = 32,
        show_progress: bool = True
    ) -> List[List[float]]:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            batch_size: Batch size for encoding
            show_progress: Show progress bar
            
        Returns:
            List of embedding vectors (each is list of floats)
        """
        if not texts:
            return []
        
        embeddings = self.model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=show_progress,
            convert_to_numpy=True,
            normalize_embeddings=True  # Recommended for bge-m3
        )
        
        return [emb.tolist() for emb in embeddings]
    
    def embed_single(self, text: str) -> List[float]:
        """Embed a single text (convenience method)"""
        return self.embed_texts([text], show_progress=False)[0]

# Test
print("="*60)
print("TEST: BGE-M3 Embedder")
print("="*60)
# embedder = BGE_M3_Embedder()  # Uncomment to test
# test_emb = embedder.embed_single("BEARING BALL TEST")
# print(f"Embedding dim: {len(test_emb)}")
# print(f"First 5 values: {test_emb[:5]}")

TEST: BGE-M3 Embedder


In [63]:
# ============================================================
# 3. INSERT DATA WITH EMBEDDINGS
# ============================================================
def insert_spare_parts(
    df: pd.DataFrame, 
    embedder: BGE_M3_Embedder, 
    batch_size: int = 128,
    start_from: int = 0
):
    """
    Insert spare part data into database with embeddings
    
    Required DataFrame columns:
        - pk_plant_matnum (PRIMARY KEY)
        - material_type, plant, material_number, plant_id, plant_description, uom
        - material_description, po_text
        - raw_combine_for_embedding, clean_combine_for_embedding
        - raw_combine_for_keyword, clean_combine_for_keyword
        - keyword_codes (list), keyword_old_codes (list), keyword_list (list)
    
    Args:
        df: DataFrame with spare part data
        embedder: BGE_M3_Embedder instance
        batch_size: Number of records to process per batch
        start_from: Skip first N records (for resuming)
    """
    
    conn = get_connection()
    cur = conn.cursor()
    
    # Normalize column names to lowercase
    df = df.copy()
    df.columns = df.columns.str.lower()
    
    # Required columns check
    required_cols = [
        'pk_plant_matnum', 'material_type', 'plant', 'material_number', 
        'plantid', 'plant_description', 'uom', 'material_description', 'text',
        'raw_combine_for_embedding', 'clean_combine_for_embedding',
        'raw_combine_for_keyword', 'clean_combine_for_keyword',
        'keyword_codes', 'keyword_old_codes', 'keyword_list'
    ]
    
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")
    
    # Prepare data
    df_to_insert = df[required_cols].copy()
    total_rows = len(df_to_insert)
    
    if start_from > 0:
        df_to_insert = df_to_insert.iloc[start_from:]
        print(f"⏭️  Skipping first {start_from} records...")
    
    print(f"📊 Total records to insert: {len(df_to_insert)}")
    
    insert_sql = f"""
        INSERT INTO {SCHEMA_NAME}.{TABLE_NAME} (
            pk_plant_matnum, material_type, plant, material_number,
            plant_id, plant_description, uom, material_description, po_text,
            raw_combine_for_embedding, clean_combine_for_embedding, embedding,
            raw_combine_for_keyword, clean_combine_for_keyword,
            keyword_codes, keyword_old_codes, keyword_list
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        ON CONFLICT (pk_plant_matnum) 
        DO UPDATE SET
            material_type = EXCLUDED.material_type,
            material_description = EXCLUDED.material_description,
            clean_combine_for_embedding = EXCLUDED.clean_combine_for_embedding,
            clean_combine_for_keyword = EXCLUDED.clean_combine_for_keyword,
            embedding = EXCLUDED.embedding,
            keyword_codes = EXCLUDED.keyword_codes,
            keyword_list = EXCLUDED.keyword_list,
            updated_at = CURRENT_TIMESTAMP
    """
    
    try:
        inserted = 0
        failed = 0
        
        # Process in batches
        for i in tqdm(range(0, len(df_to_insert), batch_size), desc="Inserting batches"):
            batch_df = df_to_insert.iloc[i:i+batch_size]
            
            # Generate embeddings for this batch
            texts_to_embed = batch_df['clean_combine_for_embedding'].fillna("").tolist()
            embeddings = embedder.embed_texts(texts_to_embed, show_progress=False)
            
            # Prepare batch data
            batch_data = []
            for idx, (_, row) in enumerate(batch_df.iterrows()):
                # Handle column name variations
                plant_id = row.get('plant_id') or row.get('plantid', '')
                po_text = row.get('po_text') or row.get('text', '')
                
                batch_data.append((
                    row['pk_plant_matnum'],
                    row['material_type'],
                    row['plant'],
                    row['material_number'],
                    plant_id,
                    row['plant_description'],
                    row['uom'],
                    row['material_description'],
                    po_text,
                    row['raw_combine_for_embedding'],
                    row['clean_combine_for_embedding'],
                    embeddings[idx],  # embedding vector
                    row['raw_combine_for_keyword'],
                    row['clean_combine_for_keyword'],
                    row['keyword_codes'] if isinstance(row['keyword_codes'], list) else [],
                    row['keyword_old_codes'] if isinstance(row['keyword_old_codes'], list) else [],
                    row['keyword_list'] if isinstance(row['keyword_list'], list) else []
                ))
            
            # Execute batch insert
            try:
                psycopg2.extras.execute_batch(cur, insert_sql, batch_data, page_size=batch_size)
                conn.commit()
                inserted += len(batch_data)
            except Exception as e:
                conn.rollback()
                failed += len(batch_data)
                print(f"\n❌ Batch {i//batch_size + 1} failed: {e}")
                continue
        
        print(f"\n✅ Insertion complete!")
        print(f"   Inserted: {inserted} records")
        if failed > 0:
            print(f"   Failed: {failed} records")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Error during insertion: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Test
print("="*60)
print("TEST: Create Table & Insert Data")
print("="*60)

import time

# # Time: Create table
# print("\n🏗️  Creating table...")
# start_time_table = time.time()
# create_spare_part_table(drop_if_exists=True)
# time_table = time.time() - start_time_table
# print(f"✅ Table created in {time_table:.2f} seconds\n")

# # Time: Insert data with embeddings
# print("📥 Inserting data with embeddings...")
# start_time_insert = time.time()
# embedder = BGE_M3_Embedder()
# insert_spare_parts(sample_300, embedder, batch_size=32)
# time_insert = time.time() - start_time_insert
# print(f"✅ Data inserted in {time_insert:.2f} seconds\n")

# print(f"⏱️  Total time: {time_table + time_insert:.2f} seconds")

TEST: Create Table & Insert Data


### 📚 Hybrid Search Methodology Summary

## 🎯 ภาพรวม Hybrid Search
ระบบใช้การรวม **4 วิธีการค้นหา** เพื่อหา material ที่คล้ายกัน:

---

## 🔍 วิธีการค้นหาทั้ง 4 แบบ

### 1️⃣ **Exact Code Match** (การจับคู่รหัสที่ตรงกัน)
- **วิธีการ**: ตรวจสอบว่า query มีรหัส (part code, drawing number) ที่ตรงกับ `keyword_codes` หรือ `keyword_list`
- **เทคนิค**: PostgreSQL Array Overlap (`&&` operator)
- **คะแนน**: 
  - 1.0 = ตรงกับ `keyword_codes` (รหัสหลัก)
  - 0.7 = ตรงกับ `keyword_list` (keyword ทั่วไป)
  - 0.0 = ไม่ตรง
- **จุดเด่น**: แม่นยำสูงมาก เหมาะกับการหารหัสที่ตรงกัน 100%

### 2️⃣ **Semantic Search** (การค้นหาตามความหมาย)
- **วิธีการ**: เปรียบเทียบ embedding vectors ระหว่าง query กับ records ใน database
- **Model**: BAAI/bge-m3 (1024 dimensions)
- **เทคนิค**: Cosine Similarity ผ่าน pgvector extension
- **คะแนน**: 0.0 - 1.0 (ยิ่งใกล้ 1 = ยิ่งคล้ายกัน)
- **จุดเด่น**: เข้าใจความหมาย เช่น "ball bearing" คล้ายกับ "rolling element bearing"

### 3️⃣ **BM25 Full-Text Search** (การค้นหาแบบคีย์เวิร์ด)
- **วิธีการ**: ค้นหาคำที่ตรงกันใน `clean_combine_for_keyword` โดยใช้ tsvector
- **เทคนิค**: PostgreSQL `ts_rank()` กับ `plainto_tsquery()`
- **คะแนน**: ขึ้นกับความถี่ของคำ และ term frequency
- **จุดเด่น**: เร็ว, ดีกับการค้นหาคำเฉพาะ (technical terms)

### 4️⃣ **Trigram Fuzzy Match** (การค้นหาแบบคล้ายๆ)
- **วิธีการ**: หาความคล้ายของ string ผ่าน trigram comparison บน `pk_plant_matnum`
- **เทคนิค**: PostgreSQL `pg_trgm` extension, `similarity()` function
- **คะแนน**: 0.0 - 1.0 (ยิ่งใกล้ 1 = string ยิ่งคล้ายกัน)
- **จุดเด่น**: รองรับ typo, เหมาะกับการหา material code ที่คล้ายๆ กัน

---

## ⚖️ Weight Profiles (น้ำหนักคะแนน)

### 🎯 **Precision Profile** (เน้นความแม่นยำ)
```python
{
    "exact_code_match": 15.0,      # ⭐ น้ำหนักสูงสุด
    "semantic_similarity": 3.0,
    "bm25_full_text": 2.0,
    "trigram_similarity": 1.0
}
```
- **เหมาะกับ**: ต้องการผลลัพธ์ที่แม่นมาก มี part code ตรงกัน
- **Use case**: หา material ที่ระบุ part number ชัดเจน

### ⚖️ **Balanced Profile** (สมดุล - **Default**)
```python
{
    "exact_code_match": 10.0,
    "semantic_similarity": 5.0,     # ⭐ เน้นความหมายมากขึ้น
    "bm25_full_text": 3.0,
    "trigram_similarity": 2.0
}
```
- **เหมาะกับ**: ใช้งานทั่วไป ต้องการทั้งความแม่นยำและความครอบคลุม
- **Use case**: ค้นหาแบบปกติ รองรับทั้งรหัสและคำอธิบาย

### 🌐 **Recall Profile** (เน้นความครอบคลุม)
```python
{
    "exact_code_match": 8.0,
    "semantic_similarity": 6.0,     # ⭐ เน้นความหมายสูงสุด
    "bm25_full_text": 5.0,          # ⭐ เน้น keyword matching
    "trigram_similarity": 4.0       # ⭐ เน้น fuzzy matching
}
```
- **เหมาะกับ**: ต้องการหาผลลัพธ์ให้มากที่สุด ยอมให้ผลลัพธ์บางตัวไม่ตรง
- **Use case**: exploratory search, หา material ที่อาจจะเกี่ยวข้อง

---

## 🧮 วิธีคำนวณ Final Score

```
Final Score = (W_code × Code_normalized) + 
              (W_sem × Semantic_normalized) + 
              (W_bm25 × BM25_normalized) + 
              (W_trigram × Trigram_normalized)
```

**ขั้นตอน**:
1. แต่ละวิธีค้นหาได้คะแนนดิบ (raw score)
2. **Normalize** คะแนนแต่ละวิธีให้อยู่ในช่วง [0, 1] โดยหารด้วย max score
3. คูณด้วย weight ตาม profile ที่เลือก
4. รวมคะแนนทั้งหมด = Final Score
5. เรียงตาม Final Score จากมากไปน้อย

---

## 💡 คำแนะนำในการปรับ Weight

- **เพิ่มน้ำหนัก Code Match** → ถ้าต้องการให้ part code มีความสำคัญมากที่สุด
- **เพิ่มน้ำหนัก Semantic** → ถ้า description มีความสำคัญ และต้องการเข้าใจความหมาย
- **เพิ่มน้ำหนัก BM25** → ถ้าต้องการจับคำเฉพาะ (specific technical terms)
- **เพิ่มน้ำหนัก Trigram** → ถ้ามี typo หรือต้องการรองรับ fuzzy matching

---

## 🚀 Use Case ตามที่อธิบาย

**โจทย์**: วน loop ทุก record เพื่อหา candidates ที่อาจเป็น material เดียวกัน แล้วส่งให้ LLM ประเมิน

**วิธีใช้**:
```python
for idx, record in df.iterrows():
    query_text = record['clean_combine_for_embedding']
    
    # หา top candidates
    candidates = hybrid_search(
        query_text=query_text,
        embedder=embedder,
        weight_profile="balanced",  # หรือ precision/recall
        limit=10  # เอา top 10 candidates
    )
    
    # ส่งให้ LLM ประเมิน
    # main_record = record
    # candidate_list = candidates
    # → LLM ตัดสินว่าตัวไหนเป็น duplicate
```

In [ ]:
# ============================================================
# 4. HYBRID SEARCH QUERY
# ============================================================

# Weight Profiles
WEIGHT_PROFILES = {
    "precision": {
        "exact_code_match": 15.0,
        "semantic_similarity": 3.0,
        "bm25_full_text": 2.0,
        "trigram_similarity": 1.0
    },
    "balanced": {
        "exact_code_match": 10.0,
        "semantic_similarity": 5.0,
        "bm25_full_text": 3.0,
        "trigram_similarity": 2.0
    },
    "recall": {
        "exact_code_match": 8.0,
        "semantic_similarity": 6.0,
        "bm25_full_text": 5.0,
        "trigram_similarity": 4.0
    }
}

def hybrid_search(
    query_text: str,
    query_text_keyword: str,  # New parameter for BM25
    embedder: BGE_M3_Embedder,
    weight_profile: str = "balanced",
    top_k_semantic: int = 20,
    top_k_bm25: int = 20,
    top_k_trigram: int = 10,
    limit: int = 40,
    uom_filter: Optional[str] = None,
    material_type_filter: Optional[str] = None
) -> pd.DataFrame:
    """
    Perform hybrid search combining semantic, keyword, and trigram matching
    
    Args:
        query_text: Search query text
        embedder: BGE_M3_Embedder instance
        weight_profile: 'precision', 'balanced', or 'recall'
        top_k_semantic: Top K results from semantic search
        top_k_bm25: Top K results from BM25
        top_k_trigram: Top K results from trigram
        limit: Final result limit
        uom_filter: Optional UOM filter (e.g., 'PC')
        material_type_filter: Optional material type filter
        
    Returns:
        DataFrame with search results and scores
    """
    
    # Get weight profile
    weights = WEIGHT_PROFILES.get(weight_profile, WEIGHT_PROFILES["balanced"])
    w_code = weights["exact_code_match"]
    w_sem = weights["semantic_similarity"]
    w_bm25 = weights["bm25_full_text"]
    w_trgm = weights["trigram_similarity"]
    
    # Generate query embedding
    query_embedding = embedder.embed_single(query_text)
    
    # Extract potential codes from query
    query_codes = extract_identifier_keywords(query_text.upper())
    
    # Extract important keywords for BM25
    # query_text_keyword is already cleaned (from clean_combine_for_keyword)
    # Just split and create OR query
    import re
    words = query_text_keyword.lower().split()
    # Clean each word: remove punctuation, keep only alphanumeric
    bm25_keywords = []
    for word in words:
        clean_word = re.sub(r'[^a-z0-9]', '', word)  # Remove all non-alphanumeric
        if len(clean_word) > 2 and clean_word not in stopwords:
            bm25_keywords.append(clean_word)
    
    # Remove duplicates while preserving order
    seen = set()
    unique_keywords = []
    for kw in bm25_keywords[:30]:  # Limit to 30 keywords
        if kw not in seen:
            seen.add(kw)
            unique_keywords.append(kw)
    
    # Create OR query string for to_tsquery
    bm25_query = ' | '.join(unique_keywords)
    
    print(f"   Extracted codes from query: {query_codes}")
    print(f"   BM25 keywords: {unique_keywords[:10]}")
    print(f"   BM25 query (OR logic): {bm25_query[:150]}...")
    print(f"   Extracted codes from query: {query_codes}")
    print(f"   BM25 keywords: {unique_keywords[:10]}")
    print(f"   BM25 query (OR logic): {bm25_query[:150]}...")
    
    conn = get_connection()
    cur = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
    print(f"   BM25 query (OR logic): {bm25_query[:150]}...")
    try:
        # Build filters
        filter_clause = ""
        if uom_filter:
            filter_clause += f" AND uom = '{uom_filter}'"
        if material_type_filter:
            filter_clause += f" AND material_type = '{material_type_filter}'"
        
        # Execute hybrid search query
        search_sql = f"""
        WITH 
        -- 1) SEMANTIC SEARCH
        semantic_search AS (
            SELECT 
                pk_plant_matnum,
                material_description,
                uom,
                (1 - (embedding <=> %s::vector)) AS semantic_score
            FROM {SCHEMA_NAME}.{TABLE_NAME}
            WHERE embedding IS NOT NULL {filter_clause}
            ORDER BY embedding <=> %s::vector
            LIMIT {top_k_semantic}
        ),
        
        -- 2) KEYWORD SEARCH (BM25) - Use to_tsquery with OR logic
        keyword_search AS (
            SELECT 
                pk_plant_matnum,
                material_description,
                uom,
                ts_rank(tsv, to_tsquery('simple', %s)) AS bm25_score
            FROM {SCHEMA_NAME}.{TABLE_NAME}
            WHERE tsv @@ to_tsquery('simple', %s)
                {filter_clause}
        ),
        
        -- 3) EXACT CODE MATCH
        exact_code_match AS (
            SELECT 
                pk_plant_matnum,
                material_description,
                uom,
                CASE 
                    WHEN keyword_codes && %s::TEXT[] THEN 1.0
                    WHEN keyword_list && %s::TEXT[] THEN 0.7
                    ELSE 0.0
                END AS code_match_score
            FROM {SCHEMA_NAME}.{TABLE_NAME}
            WHERE (keyword_codes && %s::TEXT[] OR keyword_list && %s::TEXT[]) {filter_clause}
        ),
        
        -- 4) TRIGRAM FUZZY MATCH - with lower threshold
        trigram_search AS (
            SELECT 
                pk_plant_matnum,
                material_description,
                uom,
                similarity(material_description, %s) AS trigram_score
            FROM {SCHEMA_NAME}.{TABLE_NAME}
            WHERE similarity(material_description, %s) > 0.1  -- Lower threshold
                {filter_clause}
            ORDER BY trigram_score DESC
            LIMIT {top_k_trigram}
        ),
        
        -- 5) COMBINE RESULTS
        combined_results AS (
            SELECT pk_plant_matnum, material_description, uom,
                   semantic_score, 0.0 AS bm25_score, 0.0 AS code_match_score, 0.0 AS trigram_score
            FROM semantic_search
            UNION ALL
            SELECT pk_plant_matnum, material_description, uom,
                   0.0, bm25_score, 0.0, 0.0
            FROM keyword_search
            UNION ALL
            SELECT pk_plant_matnum, material_description, uom,
                   0.0, 0.0, code_match_score, 0.0
            FROM exact_code_match
            UNION ALL
            SELECT pk_plant_matnum, material_description, uom,
                   0.0, 0.0, 0.0, trigram_score
            FROM trigram_search
        ),
        
        -- 6) AGGREGATE SCORES
        aggregated_scores AS (
            SELECT 
                pk_plant_matnum,
                MAX(material_description) AS material_description,
                MAX(uom) AS uom,
                MAX(semantic_score) AS semantic_score,
                MAX(bm25_score) AS bm25_score,
                MAX(code_match_score) AS code_match_score,
                MAX(trigram_score) AS trigram_score
            FROM combined_results
            GROUP BY pk_plant_matnum
        ),
        
        -- 7) NORMALIZE SCORES
        normalized_scores AS (
            SELECT 
                pk_plant_matnum,
                material_description,
                uom,
                CASE WHEN MAX(semantic_score) OVER () > 0 
                     THEN semantic_score / NULLIF(MAX(semantic_score) OVER (), 0)
                     ELSE 0 END AS sem_norm,
                CASE WHEN MAX(bm25_score) OVER () > 0 
                     THEN bm25_score / NULLIF(MAX(bm25_score) OVER (), 0)
                     ELSE 0 END AS bm25_norm,
                code_match_score AS code_norm,
                CASE WHEN MAX(trigram_score) OVER () > 0 
                     THEN trigram_score / NULLIF(MAX(trigram_score) OVER (), 0)
                     ELSE 0 END AS trgm_norm,
                semantic_score, bm25_score, code_match_score, trigram_score
            FROM aggregated_scores
        )
        
        -- 8) FINAL WEIGHTED SCORING
        SELECT 
            pk_plant_matnum,
            material_description,
            uom,
            ({w_code} * code_norm + {w_sem} * sem_norm + 
             {w_bm25} * bm25_norm + {w_trgm} * trgm_norm) AS final_score,
            ROUND(code_norm::numeric, 3) AS code_score,
            ROUND(sem_norm::numeric, 3) AS semantic_score,
            ROUND(bm25_norm::numeric, 3) AS bm25_score,
            ROUND(trgm_norm::numeric, 3) AS trigram_score
        FROM normalized_scores
        ORDER BY final_score DESC
        LIMIT {limit};
        """
        
        # Execute query
        cur.execute(search_sql, (
            query_embedding, query_embedding,  # semantic
            bm25_query, bm25_query,            # bm25 - OR query
            query_codes, query_codes, query_codes, query_codes,  # exact code
            query_text, query_text             # trigram
        ))
        
        results = cur.fetchall()
        
        # Convert to DataFrame
        df_results = pd.DataFrame(results)
        results = cur.fetchall()
        return df_results
        
    except Exception as e:
        print(f"❌ Search error: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Test
print("="*60)
print("TEST: Hybrid Search")
print("="*60)
# Uncomment to test with real data:
embedder = BGE_M3_Embedder()

# Get first record from sample_300 as query
query_record = sample_300.iloc[2]
print("="*60)
# Handle column name case variations
def get_col(record, col_name):
    """Get column value regardless of case"""
    col_lower = col_name.lower()
    col_title = col_name.replace('_', ' ').title().replace(' ', '_')
    return record.get(col_lower, record.get(col_title, record.get(col_name, '')))

pk = get_col(query_record, 'pk_plant_matnum')
material_desc = get_col(query_record, 'Material_Description')
uom = get_col(query_record, 'UOM')
query_text = get_col(query_record, 'clean_combine_for_embedding')  # For semantic search
query_text_keyword = get_col(query_record, 'clean_combine_for_keyword')  # For BM25

print(f"\n{'='*80}")
print("🔍 QUERY RECORD (Record #1 from sample_300)")
print(f"{'='*80}")
print(f"PK: {pk}")
print(f"Material: {material_desc}")
print(f"UOM: {uom}")
print(f"Query Text (Embedding): {query_text[:150]}...")
print(f"Query Text (Keyword): {query_text_keyword[:150]}...")

# Search for candidates
results = hybrid_search(
    query_text=query_text,
    query_text_keyword=query_text_keyword,  # Add BM25 query text
    embedder=embedder,
    weight_profile="balanced",
    limit=40
)

print(f"\n{'='*80}")
print("📊 TOP CANDIDATES (Sorted by Final Score)")
print(f"{'='*80}")
print(results[["pk_plant_matnum", "material_description", "uom", 
               "final_score", "code_score", "semantic_score", 
               "bm25_score", "trigram_score"]].to_string(index=False))

# Show top 5 in detail
print(f"\n{'='*80}")
print("🎯 TOP 5 CANDIDATES - DETAILED VIEW")
print(f"{'='*80}")
for idx, row in results.head(5).iterrows():
    print(f"\n[Rank {idx+1}] Final Score: {row['final_score']:.3f}")
    print(f"  PK: {row['pk_plant_matnum']}")
    print(f"  Description: {row['material_description']}")
    print(f"  UOM: {row['uom']}")
    print(f"  Score Breakdown:")
    print(f"    - Code Match:  {row['code_score']:.3f}")
    print(f"    - Semantic:    {row['semantic_score']:.3f}")
    print(f"    - BM25:        {row['bm25_score']:.3f}")
    print(f"    - Trigram:     {row['trigram_score']:.3f}")

TEST: Hybrid Search
🔄 Loading embedding model: BAAI/bge-m3...
✅ Model loaded! Dimension: 1024
✅ Model loaded! Dimension: 1024


NameError: name 'sample_100' is not defined

In [155]:
# Check BM25 scores in results
print("="*80)
print("BM25 Score Distribution")
print("="*80)
print(f"Total results: {len(results)}")

# Convert to numeric
results['bm25_score'] = pd.to_numeric(results['bm25_score'], errors='coerce')
results['semantic_score'] = pd.to_numeric(results['semantic_score'], errors='coerce')
results['final_score'] = pd.to_numeric(results['final_score'], errors='coerce')

print(f"\nRecords with BM25 > 0: {(results['bm25_score'] > 0).sum()}")
print(f"Records with BM25 = 0: {(results['bm25_score'] == 0).sum()}")
print(f"\nBM25 Score Statistics:")
print(results['bm25_score'].describe())
print(f"\nTop 10 by BM25 score:")
print(results.nlargest(10, 'bm25_score')[['pk_plant_matnum', 'material_description', 'bm25_score', 'semantic_score', 'final_score']].to_string())

BM25 Score Distribution
Total results: 40

Records with BM25 > 0: 40
Records with BM25 = 0: 0

BM25 Score Statistics:
count    40.000000
mean      0.888975
std       0.097800
min       0.656000
25%       0.826000
50%       0.948000
75%       0.959000
max       1.000000
Name: bm25_score, dtype: float64

Top 10 by BM25 score:
         pk_plant_matnum                   material_description  bm25_score  semantic_score  final_score
20   7560_78060663206941       BEARING,CYLINDRICAL,NUP 2206 ECP       1.000           0.000     4.822549
21   7560_78060650305951         BEARING,CYLINDRICAL,NU 305 ECJ       0.982           0.000     4.789554
4    7511_75060621642160        BEARING,DEEP GROOVE,6318,SKF(F)       0.976           0.954     7.695196
6    7813_78060617765000     BEARING,DEEP GROOVE,6305 NR,SKF(S)       0.965           0.953     7.658902
27   7813_78060617166000        BEARING,DEEP GROOVE,6320,SKF(F)       0.965           0.000     2.894666
28   7813_78060617525257        BEARING,DEEP

In [70]:
sample_300

,pk_plant_matnum,Material_Type,Plant,Material_Number,PlantID,Plant_Description,UOM,Material,Material_Description,Text,raw_combine_for_embedding,raw_combine_for_keyword,clean_combine_for_embedding,clean_combine_for_keyword,keyword_codes,keyword_old_codes,keyword_list
0,7560_75060613309212,6.0,7560,75060613309212,PT011,SKIC-Wangsa-la,PC,75060613309212,"BEARING,CYLINDRICAL,NUP 309 ECP [F]","BEARING, BORE TYPE : CYLINDRICAL, INTERNAL CL...","short_description: BEARING,CYLINDRICAL,NUP 309...","BEARING,CYLINDRICAL,NUP 309 ECP [F] BEARING, ...","short_description: bearing,cylindrical,nup 309...","bearing , cylindrical , nup 309 ecp [f] bearin...","[NUP-309, NUP309]",[nup309],"[(all), 309, NUP-309, NUP309, [f], ahlstrom, b..."
1,7511_75060621630771,6.0,7511,75060621630771,PT010,SKIC-Ban Pong,PC,75060621630771,"BEARING,DEEP GROOVE,6314-2Z/C3,SKF(F)","BEARING, BORE TYPE : CYLINDRICAL, INTERNAL CL...","short_description: BEARING,DEEP GROOVE,6314-2Z...","BEARING,DEEP GROOVE,6314-2Z/C3,SKF(F) BEARING,...","short_description: bearing,deep groove,6314-2z...","bearing , deep groove , 6314-2z/c3 , skf(f) be...",[6314-2Z/C3],[c3],"[2, 6314, 6314-2Z/C3, 6314-2z/c3, bearing, bor..."
2,7511_75060621642160,6.0,7511,75060621642160,PT010,SKIC-Ban Pong,PC,75060621642160,"BEARING,DEEP GROOVE,6318,SKF(F)","BEARING, BORE TYPE : CYLINDRICAL, INTERNAL CL...","short_description: BEARING,DEEP GROOVE,6318,SK...","BEARING,DEEP GROOVE,6318,SKF(F) BEARING, BORE ...","short_description: bearing,deep groove,6318,sk...","bearing , deep groove , 6318 , skf(f) bearing ...",[],[],"[6318, ball, bearing, bore, cage, clearance, c..."
3,7511_75060621710900,6.0,7511,75060621710900,PT010,SKIC-Ban Pong,PC,75060621710900,"BEARING,DEEP GROOVE,6206-Z,(F)","BEARING, BORE TYPE : CYLINDRICAL, INTERNAL CL...","short_description: BEARING,DEEP GROOVE,6206-Z,...","BEARING,DEEP GROOVE,6206-Z,(F) BEARING, BORE T...","short_description: bearing,deep groove,6206-z,...","bearing , deep groove , 6206-z , (f) bearing ,...",[],[],"[(f), 1, 6206, 6206-z, ball, bearing, bore, ca..."
4,7560_75060624107999,6.0,7560,75060624107999,PT011,SKIC-Wangsa-la,PC,75060624107999,"BEARING,DEEP GROOVE,6207,SKF [5]","BEARING, BORE TYPE : CYLINDRICAL, INTERNAL CL...","short_description: BEARING,DEEP GROOVE,6207,SK...","BEARING,DEEP GROOVE,6207,SKF [5] BEARING, BOR...","short_description: bearing,deep groove,6207,sk...","bearing , deep groove , 6207 , skf [5] bearing...",[],[],"[6207, [5], bearing, bore, cage, clearance, cy..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,LX11_LX06705-3123,6.0,LX11,LX06705-3123,PT028,UPPC-Philippines,PC,LX06705-3123,"*PIPE COUPLING 2"" 2 ZOLL CAST IRON KS160","Pipe Coupling 2"" 2 Zoll cast iron lamell ar gr...","short_description: *PIPE COUPLING 2"" 2 ZOLL CA...","*PIPE COUPLING 2"" 2 ZOLL CAST IRON KS160 Pipe ...","short_description: *pipe coupling 2"" 2 zoll ca...",pipe coupling 2 2 zoll cast iron ks160 pipe co...,[KS160],[ks160],"[2, 75, KS160, and, ar, cast, centr, cleaner, ..."
288,259A_25069A110128007,6.0,259A,25069A110128007,PT008,TCCB-Chonburi,PC,25069A110128007,"SEAL,SEAL KIT,WCT 4654L-00","SEAL,SEAL KIT, SHORT NAME : WCT 4654L-00 (SEA...","short_description: SEAL,SEAL KIT,WCT 4654L-00 ...","SEAL,SEAL KIT,WCT 4654L-00 SEAL,SEAL KIT, SHOR...","short_description: seal,seal kit,wct 4654l-00 ...","seal , seal kit , wct 4654l-00 seal , seal kit...",[WCT 4654L],[4654l],"[(seal, 4654l-00, 58°F, WCT 4654L, corrugator,..."
289,7580_75064711032710,6.0,7580,75064711032710,PT012,SKIC-Wangsa-la,PC,75064711032710,"PP,IMPELLER,A330 B32 Z4 APP32-125,APT [5","PP, MFR PART NAME : IMPELLER, MFR PART NO. :...","short_description: PP,IMPELLER,A330 B32 Z4 APP...","PP,IMPELLER,A330 B32 Z4 APP32-125,APT [5 PP, M...","short_description: pp,impeller,a330 b32 z4 app...","pp , impeller , a330 b32 z4 app32-125 , apt [5...","[A330, APP-32, APP32, APT32, N 92109069, PF-18]","[b32, app32-125, apt32-4, z4, a330]","[(as, 230, 32-125, 330, 92109069, A330, APP-32..."
290,OL1

In [157]:
results

,pk_plant_matnum,material_description,uom,final_score,code_score,semantic_score,bm25_score,trigram_score
0,7560_75060613309212,"BEARING,CYLINDRICAL,NUP 309 ECP [F]",PC,9.645169,0.000,0.954,0.959,1.000
1,7813_78060622909000,"BEARING,DEEP GROOVE,6209-2Z/C3,SKF(F)",PC,8.747502,0.000,0.952,0.854,0.712
2,7560_75060626111240,"BEARING,DEEP GROOVE,6311-2Z/C3,SKF [F]",PC,8.683900,0.000,0.956,0.826,0.712
3,7813_78060670409000,"BEARING,SPHERICAL,22309 E(S)",PC,7.821742,0.000,1.000,0.941,0.000
4,7511_75060621642160,"BEARING,DEEP GROOVE,6318,SKF(F)",PC,7.695196,0.000,0.954,0.976,0.000
5,7511_75060672248460,"BEARING,SPHERICAL,22318 E(F)",PC,7.674577,0.000,0.971,0.941,0.000
6,7813_78060617765000,"BEARING,DEEP GROOVE,6305 NR,SKF(S)",PC,7.658902,0.000,0.953,0.965,0.000
7,7560_75060625501999,"BEARING,DEEP GROOVE,6301,SKF [F]",PC,7.618876,0.000,0.955,0.948,0.000
8,7560_75060673219271,"BEARING,SPHERICAL,22319 E [VMI]",PC,7.610969,0.000,0.968,0.924,0.000
9,7560_75060625707999,"BEARING,DEEP GROOVE,6307-2Z,SKF [F]",PC,7.604107,0.000,0.952,0.948,0.000


### 🧪 Evaluation: Find Answer Rank in Candidates

ประเมินว่าเฉลยอยู่ที่ rank เท่าไหร่ใน candidates จาก hybrid search

In [ ]:
# ============================================================
# EVALUATION: CHECK ANSWER RANK IN CANDIDATES
# ============================================================

def evaluate_hybrid_search_quality(
    edges_df,
    sample_df,
    embedder,
    weight_profile="balanced",
    top_k=30
):
    """
    ประเมินคุณภาพของ hybrid search โดยเช็คว่าเฉลยอยู่ที่ rank เท่าไหร่
    
    Args:
        edges_df: DataFrame ที่มี Key_mat_1, Key_mat_2 (คู่ที่เป็นเฉลย)
        sample_df: DataFrame ที่มีข้อมูล material ทั้งหมด (sample_300)
        embedder: BGE_M3_Embedder instance
        weight_profile: 'precision', 'balanced', or 'recall'
        top_k: จำนวน candidates สูงสุดที่จะค้นหา
    
    Returns:
        DataFrame สรุปผลการประเมิน
    """
    
    # สร้าง mapping: pk -> คู่ของมัน (เฉลย)
    answer_map = {}
    for _, row in edges_df.iterrows():
        key1 = row['Key_mat_1']
        key2 = row['Key_mat_2']
        
        # ทั้งสองทิศทาง
        if pd.notna(key1) and pd.notna(key2):
            if key1 not in answer_map:
                answer_map[key1] = []
            if key2 not in answer_map:
                answer_map[key2] = []
            answer_map[key1].append(key2)
            answer_map[key2].append(key1)
    
    print(f"📊 Total PKs with answers: {len(answer_map)}")
    print(f"🔍 Starting evaluation with profile: {weight_profile}")
    print(f"⏱️  This may take a few minutes...\n")
    
    results = []
    
    # วน loop ทุก PK ที่มีเฉลย
    for pk, answers in tqdm(answer_map.items(), desc="Evaluating PKs"):
        # หา record นี้ใน sample_df
        record_rows = sample_df[sample_df['pk_plant_matnum'] == pk]
        
        if len(record_rows) == 0:
            continue  # skip ถ้าไม่มีใน sample
        
        record = record_rows.iloc[0]
        
        # Handle column case variations
        def get_col(rec, col_name):
            col_lower = col_name.lower()
            for col in rec.index:
                if col.lower() == col_lower:
                    return rec[col]
            return ''
        
        query_text = get_col(record, 'clean_combine_for_embedding')
        query_text_keyword = get_col(record, 'clean_combine_for_keyword')
        material_desc = get_col(record, 'material_description')
        
        if not query_text or pd.isna(query_text) or query_text == '':
            continue
        
        if not query_text_keyword or pd.isna(query_text_keyword) or query_text_keyword == '':
            continue
        
        try:
            # ทำ hybrid search
            candidates = hybrid_search(
                query_text=query_text,
                query_text_keyword=query_text_keyword,
                embedder=embedder,
                weight_profile=weight_profile,
                limit=top_k
            )
            
            # เช็คว่าเฉลยอยู่ที่ rank ไหน
            candidate_pks = candidates['pk_plant_matnum'].tolist()
            
            found_ranks = []
            for answer_pk in answers:
                if answer_pk in candidate_pks:
                    rank = candidate_pks.index(answer_pk) + 1  # rank เริ่มจาก 1
                    found_ranks.append(rank)
            
            # บันทึกผล
            if found_ranks:
                best_rank = min(found_ranks)
                status = "✅ Found"
            else:
                best_rank = None
                status = "❌ Not Found"
            
            results.append({
                'pk': pk,
                'material_description': material_desc[:50] if material_desc else '',
                'num_answers': len(answers),
                'answers': ', '.join(answers),
                'found_ranks': ', '.join(map(str, found_ranks)) if found_ranks else 'N/A',
                'best_rank': best_rank,
                'status': status
            })
            
        except Exception as e:
            print(f"\n⚠️  Error processing {pk}: {e}")
            continue
    
    # สร้าง DataFrame ผลลัพธ์
    results_df = pd.DataFrame(results)
    
    return results_df

# Test evaluation
print("="*80)
print("EVALUATION: Hybrid Search Quality Assessment")
print("="*80)
print(f"\nEvaluating {len(keys_100)} unique PKs with answers from sample50")
print("Checking at which rank the correct answer appears in candidates\n")

# import time

# # Time: Evaluation
# start_time_eval = time.time()

# embedder = BGE_M3_Embedder()  # Load model if not already loaded

# evaluation_results = evaluate_hybrid_search_quality(
#     edges_df=sample50,
#     sample_df=sample_300,
#     embedder=embedder,
#     weight_profile="balanced",
#     top_k=30
# )

# time_eval = time.time() - start_time_eval

# print(f"\n✅ Evaluation completed in {time_eval:.2f} seconds")
# print(f"⚡ Average time per query: {time_eval/len(evaluation_results):.3f} seconds\n")

# # แสดงผลลัพธ์
# print("\n" + "="*80)
# print("📊 EVALUATION RESULTS")
# print("="*80)
# print(evaluation_results[['pk', 'material_description', 'best_rank', 'status']].to_string(index=False))

# # สรุปสถิติ
# print("\n" + "="*80)
# print("📈 SUMMARY STATISTICS")
# print("="*80)

# total = len(evaluation_results)
# found = len(evaluation_results[evaluation_results['status'] == '✅ Found'])
# not_found = total - found

# print(f"Total PKs evaluated: {total}")
# print(f"Found in candidates: {found} ({found/total*100:.1f}%)")
# print(f"Not found: {not_found} ({not_found/total*100:.1f}%)")

# if found > 0:
#     found_df = evaluation_results[evaluation_results['status'] == '✅ Found']
#     print(f"\nRank Statistics (for found answers):")
#     print(f"  Mean Rank: {found_df['best_rank'].mean():.2f}")
#     print(f"  Median Rank: {found_df['best_rank'].median():.1f}")
#     print(f"  Best Rank: {found_df['best_rank'].min()}")
#     print(f"  Worst Rank: {found_df['best_rank'].max()}")
    
#     # Rank distribution
#     # print(f"\n📊 Rank Distribution:")
#     # print(f"  Top 1: {len(found_df[found_df['best_rank'] == 1])}")
#     # print(f"  Top 3: {len(found_df[found_df['best_rank'] <= 3])}")
#     # print(f"  Top 5: {len(found_df[found_df['best_rank'] <= 5])}")
#     # print(f"  Top 10: {len(found_df[found_df['best_rank'] <= 10])}")
#     # print(f"  Top 20: {len(found_df[found_df['best_rank'] <= 20])}")

#     total = len(found_df)

#     top_1 = len(found_df[found_df['best_rank'] == 1])
#     top_2_3 = len(found_df[(found_df['best_rank'] >= 2) & (found_df['best_rank'] <= 3)])
#     top_4_5 = len(found_df[(found_df['best_rank'] >= 4) & (found_df['best_rank'] <= 5)])
#     top_6_10 = len(found_df[(found_df['best_rank'] >= 6) & (found_df['best_rank'] <= 10)])
#     top_11_20 = len(found_df[(found_df['best_rank'] >= 11) & (found_df['best_rank'] <= 20)])
#     over_20 = len(found_df[found_df['best_rank'] > 20])

#     print(f"\n📊 Rank Distribution:")
#     print(f"  Top 1: {top_1}")
#     print(f"  Rank 2–3: {top_2_3}")
#     print(f"  Rank 4–5: {top_4_5}")
#     print(f"  Rank 6–10: {top_6_10}")
#     print(f"  Rank 11–20: {top_11_20}")
#     print(f"  Over 20: {over_20}")

#     print(f"\n✅ Total counted: {top_1 + top_2_3 + top_4_5 + top_6_10 + top_11_20 + over_20}")
#     print(f"📦 Total rows: {total}")

# # Show cases where answer was not found
# print("\n" + "="*80)
# print("❌ CASES WHERE ANSWER NOT FOUND")
# print("="*80)
# not_found_df = evaluation_results[evaluation_results['status'] == '❌ Not Found']
# if len(not_found_df) > 0:
#     print(not_found_df[['pk', 'material_description', 'answers']].to_string(index=False))
# else:
#     print("🎉 All answers were found in candidates!")

EVALUATION: Hybrid Search Quality Assessment

Evaluating 99 unique PKs with answers from sample50
Checking at which rank the correct answer appears in candidates



In [ ]:
# ต่อ 
# 1. insert data ทั้งหมดของ mat type 06 65k record ลง DB 
# 2. test กับ edges คำตอบ ทั้งหมด 1k record ดูว่า เฉลยอยู่ใน candidate ทั้งหมดไหม 
# 3. อย่าลืมเพิ่ม log time ที่ใช้ พร้อมกับ ลบ log ที่ไม่ใช้ 
# 4. อยากลอง test กับ ผลตัว balance / precision / recall



In [60]:
from google import genai
from google.genai import types

def generate():
    client = genai.Client(
        vertexai={
            "project": "prj-service-mlops",
            "location": "global"
        }
    )

    model = "gemini-2.5-flash"

    contents = [
        types.Content(
            role="user",
            parts=[types.Part.from_text(text="""hello""")]
        )
    ]

    tools = [
        types.Tool(google_search=types.GoogleSearch()),
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        max_output_tokens=65535,
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
        tools=tools,
        thinking_config=types.ThinkingConfig(
            thinking_budget=-1,
        ),
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if not chunk.candidates or not chunk.candidates[0].content.parts:
            continue
        print(chunk.text, end="")

generate()


Hello! How can I help you today?

In [ ]:
# Test llm-material-grouping

### 🎯 Quick Start Guide

วิธีใช้งานแบบ Step-by-Step

In [ ]:
# ============================================================
# COMPLETE WORKFLOW EXAMPLE
# ============================================================

# STEP 1: Create table (run once)
print("\n" + "="*60)
print("STEP 1: Create Table")
print("="*60)
create_spare_part_table(drop_if_exists=True)  # Set False if table exists

# STEP 2: Load embedder
print("\n" + "="*60)
print("STEP 2: Load BGE-M3 Model")
print("="*60)
embedder = BGE_M3_Embedder()

# STEP 3: Insert data
print("\n" + "="*60)
print("STEP 3: Insert Data with Embeddings")
print("="*60)

# Make sure your DataFrame has all required columns
# Example: use sample_300 or select_select_mm_matType
insert_spare_parts(
    df=sample_300,  # Replace with your DataFrame
    embedder=embedder,
    batch_size=32
)

# STEP 4: Verify data
print("\n" + "="*60)
print("STEP 4: Verify Data")
print("="*60)
conn = get_connection()
cur = conn.cursor()
cur.execute(f"SELECT COUNT(*) FROM {SCHEMA_NAME}.{TABLE_NAME}")
count = cur.fetchone()[0]
print(f"✅ Total records in table: {count}")
cur.close()
conn.close()

# STEP 5: Test search
print("\n" + "="*60)
print("STEP 5: Test Hybrid Search")
print("="*60)

# Example search
results = hybrid_search(
    query_text="BEARING BALL SKF",
    embedder=embedder,
    weight_profile="balanced",  # or 'precision' or 'recall'
    limit=10
)

print("\n📊 Search Results:")
print(results[["pk_plant_matnum", "material_description", "uom", "final_score"]].to_string())

print("\n🎉 Setup Complete!")

### 🔧 Utility Functions

Helper functions สำหรับการจัดการและ debug

In [ ]:
# # ============================================================
# # UTILITY FUNCTIONS
# # ============================================================

# def get_table_stats():
#     """Get table statistics"""
#     conn = get_connection()
#     cur = conn.cursor()
    
#     stats = {}
    
#     # Total records
#     cur.execute(f"SELECT COUNT(*) FROM {SCHEMA_NAME}.{TABLE_NAME}")
#     stats['total_records'] = cur.fetchone()[0]
    
#     # Records with embeddings
#     cur.execute(f"SELECT COUNT(*) FROM {SCHEMA_NAME}.{TABLE_NAME} WHERE embedding IS NOT NULL")
#     stats['with_embeddings'] = cur.fetchone()[0]
    
#     # Unique UOMs
#     cur.execute(f"SELECT COUNT(DISTINCT uom) FROM {SCHEMA_NAME}.{TABLE_NAME}")
#     stats['unique_uoms'] = cur.fetchone()[0]
    
#     # Unique material types
#     cur.execute(f"SELECT COUNT(DISTINCT material_type) FROM {SCHEMA_NAME}.{TABLE_NAME}")
#     stats['unique_material_types'] = cur.fetchone()[0]
    
#     cur.close()
#     conn.close()
    
#     print("📊 Table Statistics:")
#     for key, val in stats.items():
#         print(f"   {key}: {val}")
    
#     return stats

# def get_sample_records(limit: int = 5):
#     """Get sample records from table"""
#     conn = get_connection()
    
#     query = f"""
#         SELECT pk_plant_matnum, material_description, uom, 
#                array_length(keyword_codes, 1) as num_codes,
#                embedding IS NOT NULL as has_embedding
#         FROM {SCHEMA_NAME}.{TABLE_NAME}
#         LIMIT {limit}
#     """
    
#     df = pd.read_sql(query, conn)
#     conn.close()
    
#     return df

# def delete_all_records():
#     """Delete all records from table (use with caution!)"""
#     response = input(f"⚠️  Are you sure you want to delete ALL records from {SCHEMA_NAME}.{TABLE_NAME}? (yes/no): ")
    
#     if response.lower() != 'yes':
#         print("❌ Cancelled")
#         return
    
#     conn = get_connection()
#     cur = conn.cursor()
    
#     cur.execute(f"DELETE FROM {SCHEMA_NAME}.{TABLE_NAME}")
#     deleted = cur.rowcount
#     conn.commit()
    
#     cur.close()
#     conn.close()
    
#     print(f"✅ Deleted {deleted} records")

# def compare_weight_profiles(
#     query_text: str,
#     embedder: BGE_M3_Embedder,
#     limit: int = 5
# ):
#     """
#     Compare search results across different weight profiles
#     """
#     profiles = ["precision", "balanced", "recall"]
    
#     print(f"🔍 Comparing weight profiles for query: '{query_text}'\n")
    
#     all_results = {}
    
#     for profile in profiles:
#         print(f"Running {profile} profile...")
#         results = hybrid_search(
#             query_text=query_text,
#             embedder=embedder,
#             weight_profile=profile,
#             limit=limit
#         )
#         all_results[profile] = results
    
#     # Display comparison
#     print("\n" + "="*80)
#     print("RESULTS COMPARISON")
#     print("="*80)
    
#     for profile in profiles:
#         print(f"\n🎯 {profile.upper()} Profile:")
#         df = all_results[profile]
#         print(df[["pk_plant_matnum", "material_description", "final_score"]].to_string(index=False))
    
#     return all_results

# # Example usage
# print("="*60)
# print("UTILITY FUNCTIONS")
# print("="*60)
# # get_table_stats()
# # print("\nSample records:")
# # print(get_sample_records(limit=3))

In [70]:
# # Test BM25 query to see if it returns results
# test_query = "bearing angular contact cylindrical internal clearance"
# sql = f"""
# SELECT 
#     pk_plant_matnum,
#     material_description,
#     ts_rank_cd(tsv, websearch_to_tsquery('simple', %s), 32) AS bm25_score
# FROM {SCHEMA_NAME}.{TABLE_NAME}
# WHERE tsv @@ websearch_to_tsquery('simple', %s)
# ORDER BY bm25_score DESC
# LIMIT 10;
# """

# conn = get_connection()
# cur = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
# cur.execute(sql, (test_query, test_query))
# results = cur.fetchall()
# cur.close()
# conn.close()

# print(f"BM25 Test Results for query: '{test_query}'")
# print(f"Found {len(results)} results\n")
# for r in results[:5]:
#     print(f"PK: {r['pk_plant_matnum']}")
#     print(f"Description: {r['material_description']}")
#     print(f"BM25 Score: {r['bm25_score']:.6f}\n")

BM25 Test Results for query: 'bearing angular contact cylindrical internal clearance'
Found 7 results

PK: 7511_75060610300414
Description: BEARING,ANGULAR CONTACT,3310 ATN9(F)
BM25 Score: 0.041965

PK: 7511_75060610805312
Description: BEARING,ANGULAR CONTACT,7306 BECBM(F)
BM25 Score: 0.041965

PK: PO21_PO06131330172
Description: BEARING,ANGULAR CONTACT,5206 ZZ
BM25 Score: 0.034853

PK: 7560_75060602411346
Description: BEARING,ANGULAR CONTACT,3311 ANR   [F]
BM25 Score: 0.034853

PK: 7560_75060602310346
Description: BEARING,ANGULAR CONTACT,3310 ATN9  [VMI]
BM25 Score: 0.034853



### ✅ LLM Material Grouping - UPDATED VERSION

**สิ่งที่ปรับปรุง:**

1. **Output Schema ที่เรียบง่าย** - เอาเฉพาะข้อมูลที่จำเป็น:
   - `status`: "match" หรือ "unmatch" 
   - `matched_pks`: list ของ PKs ที่เป็น duplicate (ไม่รวม query_pk เอง)
   - `summary`: สรุปสั้นๆ
   
2. **Prompt ที่ชัดเจนขึ้น** - บอก LLM ให้ return เฉพาะ PKs ที่ซ้ำกัน ไม่ต้องส่ง analysis แต่ละตัว

3. **Validation ที่ถูกต้อง** - ใช้ PKs จริงจาก edges.csv:
   - 7560_75060675913234 (ไม่ใช่ 7560_75060670873346)
   - PO11_PO06131335013

In [71]:
# ============================================================
# LLM MATERIAL GROUPING
# ============================================================

from google import genai
from google.genai import types
from pydantic import BaseModel
from typing import List, Optional

# ============================================================
# Response Schema Definition
# ============================================================

class MaterialGroupingResult(BaseModel):
    """ผลลัพธ์การจัดกลุ่ม materials - เฉพาะตัวที่ซ้ำกัน"""
    query_pk: str  # PK ของ query
    status: str  # "match" = มี duplicate, "unmatch" = ไม่มี duplicate
    matched_pks: List[str]  # PKs ที่เป็น duplicate ของ query (ไม่รวม query_pk เอง)
    summary: str  # สรุปสั้นๆว่าจับคู่ได้กี่ตัว และเหตุผลหลัก

# ============================================================
# LLM Grouping Function
# ============================================================

def group_materials_with_llm(
    query_pk: str,
    query_description: str,
    query_full_text: str,
    candidates: pd.DataFrame,
    model: str = "gemini-2.0-flash-exp"
) -> MaterialGroupingResult:
    """
    ใช้ LLM วิเคราะห์และจัดกลุ่ม materials
    
    Args:
        query_pk: PK ของ material หลักที่ต้องการหา duplicate
        query_description: คำอธิบายสั้นของ query material
        query_full_text: ข้อมูลเต็มของ query material (จาก clean_combine_for_keyword)
        candidates: DataFrame ของ candidates จาก hybrid search
        model: Model name ของ Gemini
        
    Returns:
        MaterialGroupingResult object
    """
    
    # เตรียม client
    client = genai.Client(
        vertexai={
            "project": "prj-service-mlops",
            "location": "global"
        }
    )
    
    # สร้าง candidate list
    candidate_list = []
    for idx, row in candidates.iterrows():
        candidate_list.append({
            "pk": row['pk_plant_matnum'],
            "description": row['material_description'],
            "full_text": row.get('clean_combine_for_keyword', row['material_description'])[:500]  # จำกัดความยาว
        })
    
    # สร้าง prompt
    prompt = f"""# Material Deduplication Task

You are an expert in industrial spare parts and material management. Your task is to analyze whether the query material and candidate materials are **the same physical product** (duplicates/identical items).

## Query Material (The item we want to find duplicates for):
- **PK**: {query_pk}
- **Description**: {query_description}
- **Full Details**: {query_full_text[:800]}

## Candidate Materials (Potential duplicates):
"""
    
    for i, cand in enumerate(candidate_list, 1):
        prompt += f"\n### Candidate #{i}:\n"
        prompt += f"- **PK**: {cand['pk']}\n"
        prompt += f"- **Description**: {cand['description']}\n"
        prompt += f"- **Full Details**: {cand['full_text']}\n"
    
    prompt += """

## Your Task:

**Identify which candidates are DUPLICATES (same physical product) of the query material.**

## Output Requirements:

Return ONLY the PKs of candidates that are **definitely the same material** as the query:
- Set status to "match" if you find any duplicates
- Set status to "unmatch" if no duplicates found
- Include ONLY duplicate PKs in matched_pks (do NOT include the query_pk itself)
- Provide a brief summary explaining which items matched and why

## Matching Criteria:

✅ **SAME Material** (Include in matched_pks) if:
- Same manufacturer part number or model number
- Same specifications (size, type, material, etc.)
- Same physical characteristics
- Different only in: plant code, packaging, vendor code, or minor description variations

❌ **DIFFERENT Material** (Do NOT include) if:
- Different part numbers/model numbers
- Different specifications (even slightly)
- Different size, type, or material
- Different functionality or application

## Important Notes:
- Be **conservative** - only mark as duplicate if you're confident they're identical
- The first candidate is often the query itself (same PK) - DO NOT include it in matched_pks
- Different plants or vendors can have the same material
- Minor description differences don't mean different materials
- Focus on technical specifications and part numbers

Please analyze and provide your results with ONLY the duplicate PKs."""

    # เรียก LLM
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            response_mime_type="application/json",
            response_schema=MaterialGroupingResult,
            temperature=0.0  # ต้องการความ deterministic
        )
    )
    
    # Parse response
    result = MaterialGroupingResult.model_validate_json(response.text)
    
    return result

print("✅ LLM Material Grouping functions defined")

✅ LLM Material Grouping functions defined


In [72]:
# ============================================================
# TEST: LLM Grouping for PK 7813_78060670873000
# ============================================================

print("="*80)
print("TEST: LLM Material Grouping")
print("="*80)

# 1. ดึง query record
test_pk = "7813_78060670873000"
query_record = sample_300[sample_300['pk_plant_matnum'] == test_pk].iloc[0]

print(f"\n🔍 Query PK: {test_pk}")
print(f"Description: {query_record['Material_Description']}")
print(f"\nSearching for candidates...")

# 2. ทำ hybrid search
query_text = query_record['clean_combine_for_embedding']
query_text_keyword = query_record['clean_combine_for_keyword']

results = hybrid_search(
    query_text=query_text,
    query_text_keyword=query_text_keyword,
    embedder=embedder,
    weight_profile="balanced",
    limit=10  # จำกัดแค่ top 10 เพื่อไม่ให้ LLM งงมาก
)

print(f"✅ Found {len(results)} candidates\n")

# แสดง candidates
print("Top 10 Candidates:")
print("="*80)
for idx, row in results.head(10).iterrows():
    print(f"Rank {idx+1}: {row['pk_plant_matnum']}")
    print(f"  Desc: {row['material_description']}")
    print(f"  Score: {row['final_score']:.3f}")
    print()

# 3. ส่งให้ LLM วิเคราะห์
print("\n" + "="*80)
print("🤖 Sending to LLM for grouping analysis...")
print("="*80)

grouping_result = group_materials_with_llm(
    query_pk=test_pk,
    query_description=query_record['Material_Description'],
    query_full_text=query_text_keyword,
    candidates=results.head(10)
)

# 4. แสดงผลลัพธ์
print("\n" + "="*80)
print("📊 LLM GROUPING RESULTS")
print("="*80)

print(f"\nQuery PK: {grouping_result.query_pk}")
print(f"Status: {grouping_result.status.upper()}")

if grouping_result.status == "match":
    print(f"\n✅ Found {len(grouping_result.matched_pks)} duplicate(s):")
    for pk in grouping_result.matched_pks:
        # หา description ของ PK นี้จาก candidates
        matched_row = results[results['pk_plant_matnum'] == pk]
        if not matched_row.empty:
            desc = matched_row.iloc[0]['material_description']
            print(f"   - {pk}")
            print(f"     {desc}")
else:
    print("\n❌ No duplicates found")

print("\n" + "-"*80)
print("Summary:")
print("-"*80)
print(grouping_result.summary)

# 5. เช็คว่าเฉลยอยู่ใน matched_pks ไหม
actual_answers = ["7560_75060675913234", "PO11_PO06131335013"]  # จาก edges.csv
print("\n" + "="*80)
print("🎯 ANSWER VERIFICATION")
print("="*80)
print(f"Actual answers from edges.csv:")
for ans in actual_answers:
    print(f"  - {ans}")

print("\nVerification:")
all_correct = True
for ans in actual_answers:
    if ans in grouping_result.matched_pks:
        print(f"✅ {ans} FOUND in matched_pks")
    else:
        print(f"❌ {ans} NOT found in matched_pks")
        all_correct = False
        # เช็คว่า answer อยู่ใน candidates ไหม
        if ans in results['pk_plant_matnum'].values:
            rank = results[results['pk_plant_matnum'] == ans].index[0] + 1
            print(f"   (Answer IS in candidates at rank {rank})")
        else:
            print(f"   (Answer is NOT in candidates at all)")

if all_correct:
    print("\n🎉 All answers correctly identified!")
else:
    print("\n⚠️  Some answers were missed")

TEST: LLM Material Grouping

🔍 Query PK: 7813_78060670873000
Description: BEARING,SPHERICAL,22213 EK(F)

Searching for candidates...
   Extracted codes from query: ['22213-EK']
   BM25 keywords: ['bearing', 'spherical', '22213', 'ekf', 'bore', 'tapered', 'internal', 'clearance', 'normal', 'iso']
   BM25 query (OR logic): bearing | spherical | 22213 | ekf | bore | tapered | internal | clearance | normal | iso | sealshield | seal | skf | cage | material | metal | thicken...
   Extracted codes from query: ['22213-EK']
   BM25 keywords: ['bearing', 'spherical', '22213', 'ekf', 'bore', 'tapered', 'internal', 'clearance', 'normal', 'iso']
   BM25 query (OR logic): bearing | spherical | 22213 | ekf | bore | tapered | internal | clearance | normal | iso | sealshield | seal | skf | cage | material | metal | thicken...
   BM25 query (OR logic): bearing | spherical | 22213 | ekf | bore | tapered | internal | clearance | normal | iso | sealshield | seal | skf | cage | material | metal | thicken...

In [74]:
print(grouping_result)

query_pk='7813_78060670873000' status='match' matched_pks=['PO11_PO06131335013'] summary="Candidate #1 has the same PK as the query, so it's the same item but should not be included in the matched_pks. Candidate #5 (PK: PO11_PO06131335013) is a duplicate because it has the same bearing type (spherical roller) and part number (22213 EK) as the query material. The other candidates have different part numbers (e.g., 22217 EK, 22313 EK/C3, 22312 E/C3, 22314 E/C3, 22319 E, 22318 E, 22219 EK/C3) and are therefore not duplicates."


In [ ]:
# ต้องไปแก้ ไป check ดูคำตอบ edge ว่าทำไม 7511_75060672244000 ถึงไม่เจอ